# DEKAE: Dynamic Edge-driven Topology for Episodic Few-Shot Learning
## Extension of FSAKE with Supervised Relational Recovery

**Paper identity**: *A topology-learning framework with explicit structural regularization and supervised relational recovery for episodic few-shot learning.*

| Section | Description |
|---------|-------------|
| 1 | Environment Setup & GPU Check |
| 2 | Data Pipeline (Local) |
| 3 | Dataset & Episodic Loader |
| 4 | Backbone (Conv4) |
| 5 | Static k-NN Graph Module (FSAKE Baseline) |
| 6 | Edge Incidence Matrix & Dynamic Edge Features |
| 7 | Adaptive Topology Reconstruction |
| 8 | Knowledge Filtering with Edge Awareness |
| 9 | Supervised Edge Loss (Support-Support Only) |
| 10 | Sparsity Regularization |
| 11 | Full Model Assembly (DEKAE) |
| 12 | Training Loop with Checkpointing |
| **12b** | **Execute Training ← RUN THIS to train on CIFAR-FS** |
| 13 | Evaluation & Statistical Testing (dummy sanity check) |
| **13b** | **Real Test-Set Evaluation ← RUN AFTER 12b** |
| 14 | Synthetic Graph Recovery Experiment |
| 15 | Ablation Study Runner |
| **15b** | **Execute Ablation Groups A + E ← RUN AFTER 12b** |
| 16 | Metrics Logging (W&B) |
| 17 | Visualization Suite |

---
**Execution order for a full run:**
1. Run cells 1–3 (setup, data, loaders)
2. Run cells 4–12 (architecture definitions)
3. ▶ **Run cell 12b** — starts real training (~60–90 min on T4)
4. ▶ **Run cell 13b** — evaluates on test set, prints accuracy table
5. ▶ **Run cell 14** — synthetic topology recovery (trained model)
6. ▶ **Run cell 15b** — ablation sweeps (Groups A & E)


## Section 2: Data Pipeline (Local)

In [1]:
from train import RESULTS_DIR, CKPT_DIR, LOCAL_PROJECT_ROOT
# ── Section 1: Environment Setup & Dependencies ──────────────────────────────
# Run once per Colab session. Installs torch-geometric matching the Colab CUDA version.

import sys
import torch
import os

# 1. Downgrade setuptools to avoid build errors with older packages
!pip install -q "setuptools<70" wheel Cython

# 2. Install standard dependencies
!pip install -q torch-geometric wandb scipy scikit-learn matplotlib seaborn networkx tqdm Pillow

# 3. Install learn2learn from source in editable mode (bypasses wheel build failure)
if not os.path.exists("learn2learn"):
    !git clone https://github.com/learnables/learn2learn.git

# Install with -e (editable) to skip bdist_wheel
!cd learn2learn && pip install -e .

print("PyTorch version :", torch.__version__)
print("CUDA available  :", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU             :", torch.cuda.get_device_name(0))
    print("VRAM            :", round(torch.cuda.get_device_properties(0).total_memory / 1e9, 2), "GB")
else:
    print("⚠ No GPU detected — switch Runtime → Change runtime type → GPU")

# ── Reproducibility ───────────────────────────────────────────────────────────
import random, numpy as np

GLOBAL_SEED = 42

def set_seed(seed: int = GLOBAL_SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

Obtaining file:///e:/projects/few_shot_gnn/learn2learn
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Using cached gym-0.26.2-py3-none-any.whl
  Using cached gsutil-5.31-py3-none-any.whl
  Using cached cloudpickle-3.1.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached gym_notices-0.1.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached torch-2.9.1-cp314-cp314-win_amd64.whl.metadata (30 kB)
  Using cached argcomplete-3.6.3-py3-none-any.whl.

  error: subprocess-exited-with-error
  
  Building editable for learn2learn (pyproject.toml) did not run successfully.
  exit code: 1
  
  [22 lines of output]
  running editable_wheel
  creating C:\Users\Administrator\AppData\Local\Temp\3\pip-ephem-wheel-cache-1f5owc4h\wheels\57\ac\8e\6dda8656dee790eb13e537e5e826fe268386f78ffc399431a0\tmpoyy892lj\.tmp-yg3fp_yg\learn2learn.egg-info
  writing C:\Users\Administrator\AppData\Local\Temp\3\pip-ephem-wheel-cache-1f5owc4h\wheels\57\ac\8e\6dda8656dee790eb13e537e5e826fe268386f78ffc399431a0\tmpoyy892lj\.tmp-yg3fp_yg\learn2learn.egg-info\PKG-INFO
  writing dependency_links to C:\Users\Administrator\AppData\Local\Temp\3\pip-ephem-wheel-cache-1f5owc4h\wheels\57\ac\8e\6dda8656dee790eb13e537e5e826fe268386f78ffc399431a0\tmpoyy892lj\.tmp-yg3fp_yg\learn2learn.egg-info\dependency_links.txt
  writing requirements to C:\Users\Administrator\AppData\Local\Temp\3\pip-ephem-wheel-cache-1f5owc4h\wheels\57\ac\8e\6dda8656dee790eb13e537e5e826fe268386f78ffc399431a

## Section 2: Google Drive Mount & Data Pipeline

In [2]:
# ── Section 2: Data Download + Local Project Setup ───────────────────────────
# Strategy:
#   • Datasets are downloaded automatically via learn2learn (no manual work).
#   • Downloaded data lives on /content/data  (fast local SSD).
#   • Checkpoints & results are saved to /content/FSAKE_Project (local SSD).
#     They persist for the lifetime of the session. Download anything you want
#     to keep before the runtime resets (Files panel → right-click → Download).
#     No Google Drive required.
#
# Supported datasets (all download automatically on first run):
#   ACTIVE_DATASET = 'miniImageNet'   ← change to switch dataset
#   Other options : 'tieredImageNet' | 'cifarFS' | 'cub'

import shutil, pathlib, zipfile, sys, os

# ── Fix Import Shadowing ─────────────────────────────────────────────────────
# The repo is at /content/learn2learn. The package is /content/learn2learn/learn2learn.
# If CWD is /content, 'import learn2learn' finds the repo folder (namespace pkg) first.

# 1. Force reload if already loaded incorrectly
if "learn2learn" in sys.modules:
    import learn2learn
    # If it lacks __file__, it's likely the namespace package (broken)
    if not hasattr(learn2learn, "__file__") or learn2learn.__file__ is None:
        print("Creating fix: unloading broken learn2learn module...")
        del sys.modules["learn2learn"]

# 2. Prepend repo path to sys.path so the inner package is found first
if os.path.exists("/content/learn2learn"):
    if "/content/learn2learn" not in sys.path:
        sys.path.insert(0, "/content/learn2learn")

import learn2learn as l2l
print(f"learn2learn imported from: {getattr(l2l, '__file__', 'unknown')}")

# ─────────────────────────────────────────────────────────────────────────────

# SWITCHED TO cifarFS for reliability (miniImageNet download is often unstable)
ACTIVE_DATASET  = 'cifarFS'
LOCAL_DATA_ROOT = pathlib.Path("/content/data")
LOCAL_DATA_ROOT.mkdir(parents=True, exist_ok=True)

# ── Step 1: Download dataset via learn2learn ─────────────────────────────────
from learn2learn.vision.datasets import (
    MiniImagenet, TieredImagenet, CIFARFS, CUBirds200
)

from torchvision import transforms

MEAN = [0.4712, 0.4499, 0.4031]
STD  = [0.2726, 0.2634, 0.2794]

BASE_TRANSFORM = transforms.Compose([
    transforms.Resize(84),
    transforms.CenterCrop(84),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD),
])
AUG_TRANSFORM = transforms.Compose([
    transforms.RandomResizedCrop(84),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.4, 0.4, 0.4, 0.1),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD),
])

DATASET_CLS = {
    'miniImageNet' : MiniImagenet,
    'tieredImageNet': TieredImagenet,
    'cifarFS'      : CIFARFS,
    'cub'          : CUBirds200,
}

print(f"Downloading / loading {ACTIVE_DATASET} …")
cls = DATASET_CLS[ACTIVE_DATASET]

# download=True fetches the data automatically on first run.
# Subsequent runs reuse the cached copy in LOCAL_DATA_ROOT.
try:
    train_dataset = cls(root=str(LOCAL_DATA_ROOT), mode='train',
                        download=True, transform=BASE_TRANSFORM)
    val_dataset   = cls(root=str(LOCAL_DATA_ROOT), mode='validation',
                        download=True, transform=BASE_TRANSFORM)
    test_dataset  = cls(root=str(LOCAL_DATA_ROOT), mode='test',
                        download=True, transform=BASE_TRANSFORM)

    # Wrap in MetaDataset to ensure labels_to_indices exists (needed for EpisodicSampler)
    train_dataset = l2l.data.MetaDataset(train_dataset)
    val_dataset   = l2l.data.MetaDataset(val_dataset)
    test_dataset  = l2l.data.MetaDataset(test_dataset)

    print(f"{ACTIVE_DATASET} — "
          f"train: {len(train_dataset.labels_to_indices)} classes, "
          f"val: {len(val_dataset.labels_to_indices)} classes, "
          f"test: {len(test_dataset.labels_to_indices)} classes")
    print(f"Data stored at : {LOCAL_DATA_ROOT}")

except Exception as e:
    print(f"\n❌ Error downloading {ACTIVE_DATASET}: {e}")
    print("Tip: If miniImageNet fails, try switching ACTIVE_DATASET to 'cifarFS' or 'tieredImageNet'.")
    raise e

# ── Step 2: Local checkpoint / results directories ────────────────────────────
# Saves everything to the Colab instance SSD — no Google Drive needed.
# To keep results after the session ends, use the Files panel (left sidebar)
# to download checkpoints/results before the runtime resets.
LOCAL_PROJECT_ROOT = pathlib.Path("/content/FSAKE_Project")

CKPT_DIR    = LOCAL_PROJECT_ROOT / "checkpoints"
RESULTS_DIR = LOCAL_PROJECT_ROOT / "results"
CKPT_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print("CKPT dir    :", CKPT_DIR)
print("Results dir :", RESULTS_DIR)


learn2learn imported from: None


ModuleNotFoundError: No module named 'learn2learn.vision'

## Section 3: Dataset & Episodic Loader

In [3]:
# ── Section 3: Dataset & Episodic Loader ─────────────────────────────────────
# Wraps the learn2learn datasets (loaded in Section 2) into a simple episodic
# sampler. Works with miniImageNet, tieredImageNet, CIFAR-FS, and CUB.
#
# Episodic protocol:
#   S = N_way × K_shot  support samples  (labeled)
#   Q = N_way × N_query query  samples  (unlabeled at inference)

import torch
import torch.nn.functional as F

# ── Generic episodic sampler wrapping any learn2learn dataset ─────────────────

class EpisodicSampler:
    """
    Wraps a learn2learn dataset (which exposes `labels_to_indices` dict)
    and samples N-way K-shot episodes on demand.

    Usage:
        sampler = EpisodicSampler(train_dataset, n_way=5, k_shot=1, n_query=15)
        s_imgs, s_lbl, q_imgs, q_lbl = sampler.sample()   # one episode

    Can also be called as a function (for compatibility with train/eval loops):
        s_imgs, s_lbl, q_imgs, q_lbl = sampler(n_way=5, k_shot=1, n_query=15)
    """

    def __init__(self, dataset, n_way: int = 5, k_shot: int = 1,
                 n_query: int = 15):
        self.dataset  = dataset
        self.n_way    = n_way
        self.k_shot   = k_shot
        self.n_query  = n_query

        # Build class → list[global_index] map
        if hasattr(dataset, 'labels_to_indices'):
            self.class_to_indices = {
                c: list(idxs)
                for c, idxs in dataset.labels_to_indices.items()
                if len(idxs) >= k_shot + n_query
            }
        elif hasattr(dataset, 'y'):
            import numpy as _np
            _labels = _np.array(dataset.y)
            self.class_to_indices = {
                int(c): list(_np.where(_labels == c)[0])
                for c in _np.unique(_labels)
                if len(_np.where(_labels == c)[0]) >= k_shot + n_query
            }
        else:
            raise ValueError("Dataset must expose .labels_to_indices or .y")

        self.classes = sorted(self.class_to_indices.keys())
        assert len(self.classes) >= n_way, (
            f"Only {len(self.classes)} classes available, N-way={n_way}")
        print(f"EpisodicSampler ready: {len(self.classes)} classes, "
              f"{n_way}-way {k_shot}-shot {n_query}-query")

    def sample(self):
        """Returns (support_imgs, support_labels, query_imgs, query_labels)."""
        episode_classes = random.sample(self.classes, self.n_way)
        s_imgs, s_labels, q_imgs, q_labels = [], [], [], []
        for local_lbl, cls in enumerate(episode_classes):
            pool = random.sample(self.class_to_indices[cls],
                                 self.k_shot + self.n_query)
            for i, idx in enumerate(pool):
                img, _ = self.dataset[idx]       # transform applied by dataset
                if not isinstance(img, torch.Tensor):
                    from torchvision.transforms.functional import to_tensor
                    img = to_tensor(img)
                if i < self.k_shot:
                    s_imgs.append(img);   s_labels.append(local_lbl)
                else:
                    q_imgs.append(img);   q_labels.append(local_lbl)
        return (torch.stack(s_imgs),  torch.tensor(s_labels, dtype=torch.long),
                torch.stack(q_imgs),  torch.tensor(q_labels, dtype=torch.long))

    def __call__(self, n_way=None, k_shot=None, n_query=None):
        """Allow calling as episode_fn(n_way, k_shot, n_query) for loop compat."""
        if n_way   is not None: self.n_way   = n_way
        if k_shot  is not None: self.k_shot  = k_shot
        if n_query is not None: self.n_query = n_query
        return self.sample()


# ── Instantiate samplers for all three splits ─────────────────────────────────
train_sampler = EpisodicSampler(train_dataset, n_way=5, k_shot=1, n_query=15)
val_sampler   = EpisodicSampler(val_dataset,   n_way=5, k_shot=1, n_query=15)
test_sampler  = EpisodicSampler(test_dataset,  n_way=5, k_shot=1, n_query=15)

# Augmented training sampler (same data, richer color/crop augmentation)
train_dataset_aug = DATASET_CLS[ACTIVE_DATASET](
    root=str(LOCAL_DATA_ROOT), mode='train',
    download=False, transform=AUG_TRANSFORM)
# FIX: Wrap train_dataset_aug with l2l.data.MetaDataset
train_dataset_aug = l2l.data.MetaDataset(train_dataset_aug)
train_sampler_aug = EpisodicSampler(train_dataset_aug, n_way=5, k_shot=1, n_query=15)

# ── Quick episode sanity check ────────────────────────────────────────────────
_s_imgs, _s_lbl, _q_imgs, _q_lbl = train_sampler.sample()
print(f"Support : {_s_imgs.shape}  labels={_s_lbl.tolist()}")
print(f"Query   : {_q_imgs.shape}  labels={_q_lbl[:5].tolist()}…")

# ── Synthetic planted-partition episode generator (Section 2.3) ───────────────

def synthetic_episode(n_way=5, n_nodes_per_class=5, feat_dim=64,
                      sigma=0.8, device=DEVICE):
    """
    Generate a planted-partition episode for topology-recovery experiments.
    Returns node features X, labels y, and ground-truth adjacency A_star.
    sigma controls noise: larger sigma → harder topology recovery.
    """
    centers = torch.randn(n_way, feat_dim)
    center_dists = [
        (centers[i] - centers[j]).norm().item()
        for i in range(n_way)
        for j in range(i + 1, n_way)
    ]
    avg_sep = sum(center_dists) / len(center_dists)

    N = n_way * n_nodes_per_class
    X = torch.zeros(N, feat_dim)
    y = torch.zeros(N, dtype=torch.long)
    for c in range(n_way):
        sl = slice(c * n_nodes_per_class, (c + 1) * n_nodes_per_class)
        X[sl] = centers[c] + sigma * avg_sep * torch.randn(n_nodes_per_class, feat_dim)
        y[sl] = c

    # Ground-truth: intra-class = 1, inter-class = 0, no self-loops
    A_star = (y.unsqueeze(0) == y.unsqueeze(1)).float()
    A_star.fill_diagonal_(0)
    return X.to(device), y.to(device), A_star.to(device)


# ── Quick sanity check ────────────────────────────────────────────────────────
X_syn, y_syn, A_syn = synthetic_episode(n_way=5, n_nodes_per_class=4, feat_dim=64, sigma=0.6)
print(f"Synthetic X:{X_syn.shape}  y:{y_syn.shape}  A*:{A_syn.shape}")
print(f"Intra-class edges: {int(A_syn.sum().item())} "
      f"(expected {5 * 4 * 3} for 5 classes, 4 nodes each)")

NameError: name 'train_dataset' is not defined

## Section 4: Backbone Network (Conv4)
Bit-for-bit identical to the FSAKE backbone. 128-dim output embedding.

In [4]:
# ── Section 4: Backbone Network (Conv4) ──────────────────────────────────────
import torch.nn as nn

def conv_block(in_ch: int, out_ch: int) -> nn.Sequential:
    """Standard FSAKE conv block: Conv → BN → ReLU → MaxPool."""
    return nn.Sequential(
        nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, bias=False),
        nn.BatchNorm2d(out_ch),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2),
    )


class Conv4(nn.Module):
    """
    4-layer convolutional backbone identical to FSAKE.
    Input : (B, 3, 84, 84)
    Output: (B, embed_dim) — default 128 to match FSAKE paper.
    """

    def __init__(self, in_channels: int = 3, embed_dim: int = 128):
        super().__init__()
        self.encoder = nn.Sequential(
            conv_block(in_channels, 64),   # → 42×42
            conv_block(64, 64),            # → 21×21
            conv_block(64, 64),            # → 10×10
            conv_block(64, 64),            # →  5× 5
        )
        self.proj = nn.Linear(64 * 5 * 5, embed_dim)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        h = self.encoder(x)                          # (B, 64, 5, 5)
        h = h.view(h.size(0), -1)                    # (B, 1600)
        return self.proj(h)                          # (B, embed_dim)


# ── Sanity check ─────────────────────────────────────────────────────────────
_backbone = Conv4(embed_dim=128).to(DEVICE)
_dummy    = torch.randn(10, 3, 84, 84).to(DEVICE)
_out      = _backbone(_dummy)
print("Conv4 output shape:", _out.shape)   # expect (10, 128)
assert _out.shape == (10, 128), "Backbone shape mismatch!"
total_params = sum(p.numel() for p in _backbone.parameters() if p.requires_grad)
print(f"Conv4 trainable parameters: {total_params:,}")

Conv4 output shape: torch.Size([10, 128])
Conv4 trainable parameters: 317,760


## Section 5: Static k-NN Graph Module (FSAKE Baseline)

In [5]:
# ── Section 5: Static k-NN Graph Module (FSAKE Baseline) ─────────────────────
# Builds A ∈ R^{N×N} from pairwise Euclidean distances; retains top-k per node.
# This is the graph construction used by FSAKE — plug-in swappable.

def build_knn_adjacency(node_feats: torch.Tensor, k: int,
                        symmetric: bool = True) -> torch.Tensor:
    """
    Args
    ----
    node_feats : (N, d) node embeddings
    k          : number of nearest neighbours per node
    symmetric  : if True, A = max(A, A^T) (undirected graph)

    Returns
    -------
    A : (N, N) float adjacency (0/1 values, no self-loops, row-normalised)
    """
    N = node_feats.size(0)
    # Pairwise squared Euclidean distance
    diff = node_feats.unsqueeze(0) - node_feats.unsqueeze(1)   # (N,N,d)
    dist = (diff ** 2).sum(-1)                                  # (N,N)

    # Mask self (set diagonal to large value so it is not selected in top-k)
    dist_masked = dist.clone()
    dist_masked.fill_diagonal_(float("inf"))

    # Keep k SMALLEST distances (nearest neighbours) → set all others to 0
    _, topk_idx = dist_masked.topk(k, dim=1, largest=False)
    A = torch.zeros(N, N, dtype=torch.float32, device=node_feats.device)
    A.scatter_(1, topk_idx, 1.0)

    if symmetric:
        A = torch.max(A, A.t())

    # Row-normalise (D^{-1} A)
    deg = A.sum(dim=1, keepdim=True).clamp(min=1e-6)
    A_norm = A / deg
    return A_norm


# ── Graph utility: density ────────────────────────────────────────────────────

def graph_density(A: torch.Tensor) -> float:
    """Fraction of possible directed edges that are non-zero (excl. diagonal)."""
    N = A.size(0)
    possible = N * (N - 1)
    actual   = (A > 0).float().sum().item() - (A.diagonal() > 0).float().sum().item()
    density  = actual / possible if possible > 0 else 0.0
    if density > 0.5:
        print(f"⚠  Graph density = {density:.3f} > 0.5 — risk of topology collapse!")
    return density


# ── Sanity check ─────────────────────────────────────────────────────────────
_feats = torch.randn(20, 128).to(DEVICE)
_A_knn = build_knn_adjacency(_feats, k=5)
d = graph_density(_A_knn)
print(f"k-NN adjacency shape: {_A_knn.shape}  density: {d:.3f}")

k-NN adjacency shape: torch.Size([20, 20])  density: 0.384


## Section 6: Edge Incidence Matrix & Dynamic Edge Features

**Key distinction from attention**: Attention produces a *scalar* weight per edge. Computing $E = BXW$ via the incidence matrix $B \in \{0,1\}^{N \times |E|}$ produces a *vector* $e_{ij} \in \mathbb{R}^d$ per edge — a $d$-dimensional signal retaining directional and heterogeneous relational information. This is equivalent to one step of line-graph convolution, where edges are first-class computational objects.

In [6]:
# ── Section 6: Edge Incidence Matrix & Dynamic Edge Features ─────────────────
# B ∈ {0,1}^{N × |E|}  maps each edge to its two incident nodes.
# E = B^T X W  →  each edge gets a vector representation in R^d.

class EdgeIncidenceModule(nn.Module):
    """
    Builds the edge incidence matrix B from a given adjacency A,
    then computes edge features E = B^T X W using a shallow MLP.

    Parameters
    ----------
    in_dim  : node feature dimension d
    edge_dim: output edge feature dimension (default = in_dim for skip-compat.)
    hidden  : hidden dim of the 2-layer edge MLP (≤ 64 to avoid overfit)
    """

    def __init__(self, in_dim: int, edge_dim: int = None, hidden: int = 64):
        super().__init__()
        edge_dim = edge_dim or in_dim
        self.edge_mlp = nn.Sequential(
            nn.Linear(2 * in_dim, hidden, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(hidden, edge_dim, bias=True),
        )

    @staticmethod
    def adjacency_to_edge_index(A: torch.Tensor):
        """Return (src, dst) index arrays for all non-zero entries of A."""
        nz = A.nonzero(as_tuple=True)
        return nz[0], nz[1]         # src, dst

    def forward(self, X: torch.Tensor, A: torch.Tensor):
        """
        X : (N, d)  node features
        A : (N, N)  adjacency (may be weighted)

        Returns
        -------
        E         : (|E|, edge_dim)  vector edge features
        src, dst  : edge index tensors
        """
        src, dst = self.adjacency_to_edge_index(A)
        # Concatenate incident node features for each edge
        edge_input = torch.cat([X[src], X[dst]], dim=-1)    # (|E|, 2d)
        E = self.edge_mlp(edge_input)                        # (|E|, edge_dim)
        return E, src, dst


# ── Sanity check ─────────────────────────────────────────────────────────────
_eim   = EdgeIncidenceModule(in_dim=128, edge_dim=128, hidden=64).to(DEVICE)
_feats = torch.randn(20, 128).to(DEVICE)
_A     = build_knn_adjacency(_feats, k=5)
_E, _src, _dst = _eim(_feats, _A)
print(f"Edge features shape : {_E.shape}   (|E|={len(_src)}, edge_dim=128)")
print(f"src index range     : {_src.min().item()} – {_src.max().item()}")

Edge features shape : torch.Size([134, 128])   (|E|=134, edge_dim=128)
src index range     : 0 – 19


## Section 7: Adaptive Topology Reconstruction

Low-rank bilinear scoring: $s_{ij} = (W_1 h_i)^\top (W_2 h_j)$ with rank $r \ll d$ bounds overfitting. Spectral normalization bounds $L_f$ (Lipschitz). Row-normalisation of $A'$ bounds $L_{MP}$. Together they ensure contraction (Proposition 3).

In [7]:
# ── Section 7: Adaptive Topology Reconstruction ───────────────────────────────
from torch.nn.utils.parametrizations import spectral_norm as spectral_norm_wrap

class DynamicTopologyModule(nn.Module):
    """
    Learns A'(l) = f(H(l)) using low-rank bilinear scoring.

    s_ij = (W1 h_i)^T (W2 h_j) / sqrt(r)     [low-rank, rank r << d]

    Spectral normalization on W1,W2 → Lipschitz bound on f.
    Row-normalization of A'          → Lipschitz bound on MP.

    sparsity_mode ∈ {'none', 'l1', 'topk', 'laplacian'}
    """

    def __init__(self, feat_dim: int, rank: int = 16, edge_dim: int = 128,
                 sparsity_mode: str = "l1", topk: int = 5,
                 lambda_sparse: float = 0.01, edge_dropout: float = 0.1):
        super().__init__()
        self.rank          = rank
        self.sparsity_mode = sparsity_mode
        self.topk          = topk
        self.lambda_sparse = lambda_sparse
        self.edge_dropout  = edge_dropout

        # Low-rank projection (spectral-normalised)
        self.W1 = spectral_norm_wrap(nn.Linear(feat_dim, rank, bias=False))
        self.W2 = spectral_norm_wrap(nn.Linear(feat_dim, rank, bias=False))

        # Edge → A' scoring via edge feature dot-product
        self.edge_scorer = spectral_norm_wrap(
            nn.Linear(edge_dim, 1, bias=False))

    def forward(self, H: torch.Tensor, E: torch.Tensor,
                src: torch.Tensor, dst: torch.Tensor,
                sparsity_reg: bool = True):
        """
        H   : (N, d)      node features
        E   : (|E|, d_e)  edge features from EdgeIncidenceModule
        src,dst : edge indices

        Returns
        -------
        A_prime       : (N, N)  new normalised adjacency
        sparsity_loss : scalar  (0 if sparsity_reg=False)
        """
        N = H.size(0)

        # ── Low-rank bilinear scores ─────────────────────────────────────────
        scores_bilinear = (self.W1(H[src]) * self.W2(H[dst])).sum(-1) / (self.rank ** 0.5)

        # ── Edge-feature scores ──────────────────────────────────────────────
        scores_edge     = self.edge_scorer(E).squeeze(-1)

        edge_scores = scores_bilinear + scores_edge

        # ── Dropout on edge weights during training ──────────────────────────
        if self.training and self.edge_dropout > 0:
            mask = torch.bernoulli(
                torch.full_like(edge_scores, 1 - self.edge_dropout))
            edge_scores = edge_scores * mask

        # ── Soft adjacency via sigmoid ────────────────────────────────────────
        edge_weights = torch.sigmoid(edge_scores)

        # ── Build dense A' ────────────────────────────────────────────────────
        A_prime = torch.zeros(N, N, device=H.device, dtype=H.dtype)
        A_prime[src, dst] = edge_weights

        # ── Sparsity regularisation ───────────────────────────────────────────
        sparsity_loss = torch.tensor(0.0, device=H.device)
        if sparsity_reg:
            if self.sparsity_mode == "l1":
                sparsity_loss = self.lambda_sparse * A_prime.abs().mean()
            elif self.sparsity_mode == "topk":
                # Hard top-k mask (gradient stops here — ablation only)
                vals, idx = A_prime.topk(self.topk, dim=1)
                mask = torch.zeros_like(A_prime)
                mask.scatter_(1, idx, 1.0)
                A_prime = A_prime * mask.detach()
            elif self.sparsity_mode == "laplacian":
                # tr(H^T L H) where L = D - A'
                D = A_prime.sum(dim=1).diag()
                L = D - A_prime
                sparsity_loss = self.lambda_sparse * torch.trace(H.t() @ L @ H)

        # ── Row-normalise (Proposition 3 Lipschitz control) ──────────────────
        deg = A_prime.sum(dim=1, keepdim=True).clamp(min=1e-6)
        A_prime = A_prime / deg

        return A_prime, sparsity_loss


def graph_density(A: torch.Tensor) -> float:
    """
    Quick graph density helper (used in early sanity checks before
    graph_metrics() is defined in Section 10).
    graph_density = |E| / (N*(N-1))  (self-loops excluded).
    """
    N = A.size(0)
    A_no_diag = A.clone()
    A_no_diag.fill_diagonal_(0)
    n_edges = (A_no_diag > 0).float().sum().item()
    return n_edges / max(N * (N - 1), 1)


# ── Sanity check ─────────────────────────────────────────────────────────────
_dtm   = DynamicTopologyModule(feat_dim=128, rank=16, edge_dim=128).to(DEVICE)
_A_dyn, _sl = _dtm(_feats, _E, _src, _dst)
print(f"Dynamic A' shape: {_A_dyn.shape}  sparsity_loss={_sl.item():.4f}")
print(f"Graph density   : {graph_density(_A_dyn):.3f}")   # uses helper defined above


Dynamic A' shape: torch.Size([20, 20])  sparsity_loss=0.0017
Graph density   : 0.353


## Section 8: Edge-Aware Node Update — Case 1 & Case 2

**Case 1 (node-aggregation with edge gating)** — `EdgeAwareKnowledgeFilter`:  
$X'_i = \sigma(g(e_i)) \cdot \sum_j A'_{ij} h_j$  
Edge features gate importance scores, but the node update is still a weighted sum of *neighboring node features*.

**Case 2 — the stronger novelty** — `EdgeToNodeProjection`:  
$$X' = A_{edge} \cdot E \cdot W', \quad A_{edge} \in \mathbb{R}^{N \times |E|}$$  
Each node is updated from **edge feature vectors** directly. $A_{edge}[i, e]$ holds a learned similarity weight for edge $e$ at its incident node $i$, computed as $\sigma(\text{MLP}([e_{ij} \| h_i]))$. This is structurally impossible with scalar attention and is equivalent to one step of line-graph message passing — exactly what novelty2.md calls the "stronger novelty" of Case 2.

Both modules share the same interface `(H, E, src, dst) → (scores, H_new)` so they are plug-in swappable via the `use_case2` flag in `DEKAEModel`.


In [8]:
# ── Section 8: Edge-Aware Node Update ────────────────────────────────────────

# ─────────────────────────────────────────────────────────────────────────────
# Case 1: EdgeAwareKnowledgeFilter
#   Node update = weighted sum of NEIGHBORING NODE FEATURES, gated by edge agg.
#   X'_i = σ(g(e_i)) · Σ_j A'_ij h_j
# ─────────────────────────────────────────────────────────────────────────────
class EdgeAwareKnowledgeFilter(nn.Module):
    """
    Case 1: classic node-aggregation gated by aggregated edge features.
    Kept for ablation baseline (A3 path).
    """

    def __init__(self, edge_dim: int, node_dim: int, hidden: int = 64):
        super().__init__()
        self.gate = nn.Sequential(
            nn.Linear(edge_dim + node_dim, hidden),
            nn.ReLU(inplace=True),
            nn.Linear(hidden, 1),
        )

    def forward(self, H: torch.Tensor, E: torch.Tensor,
                src: torch.Tensor, dst: torch.Tensor,
                A_prime: torch.Tensor):
        """
        H       : (N, d)
        E       : (|E|, edge_dim)
        src,dst : edge endpoint indices
        A_prime : (N, N) row-normalised dynamic adjacency

        Returns
        -------
        scores   : (N,) importance scores in [0,1]
        H_pooled : (N, d) node update via A' @ H, gated by edge aggregation
        """
        N        = H.size(0)
        edge_dim = E.size(1)

        # Mean-aggregate incident edge features per node
        agg = torch.zeros(N, edge_dim, device=H.device)
        agg.scatter_add_(0, src.unsqueeze(-1).expand_as(E), E)
        counts = torch.zeros(N, device=H.device).scatter_add_(
            0, src, torch.ones(len(src), device=H.device))
        agg = agg / counts.unsqueeze(-1).clamp(min=1)

        gate_input = torch.cat([agg, H], dim=-1)
        scores     = torch.sigmoid(self.gate(gate_input)).squeeze(-1)  # (N,)

        H_pooled   = A_prime @ H                             # Case 1: node→node
        H_pooled   = H_pooled * scores.unsqueeze(-1)
        return scores, H_pooled


# ─────────────────────────────────────────────────────────────────────────────
# Case 2: EdgeToNodeProjection  ← THE STRONGER NOVELTY (novelty2.md §4, Case 2)
#   Node update = weighted sum of EDGE FEATURE VECTORS
#   X' = A_edge @ E @ W'   where A_edge ∈ R^{N × |E|}
#
#   A_edge[i, e] = σ( MLP([e_ij ‖ h_i]) )  — learned similarity-based weight
#                                              non-zero only when i ∈ {src_e, dst_e}
#
#   Structural difference from Case 1 / attention:
#     • Message source is a VECTOR e_{ij} ∈ R^d, not a scalar weight
#     • Equivalent to one step of line-graph message passing
#     • Enables edge-level loss supervision (structurally impossible with scalars)
# ─────────────────────────────────────────────────────────────────────────────
class EdgeToNodeProjection(nn.Module):
    """
    Case 2: X' = A_edge @ E @ W'

    For each edge e = (i, j):
      - Compute assignment weight w_i = σ( scorer([e ‖ h_i]) )  at node i (src)
      - Compute assignment weight w_j = σ( scorer([e ‖ h_j]) )  at node j (dst)
      - Scatter: H'_i += w_i · (E_e @ W');  H'_j += w_j · (E_e @ W')

    The result H'_i is a weighted sum of *projected edge feature vectors*,
    not of neighboring node features — this is the key structural distinction.

    Parameters
    ----------
    edge_dim : edge feature dimension (from EdgeIncidenceModule)
    node_dim : output node feature dimension
    """

    def __init__(self, edge_dim: int, node_dim: int):
        super().__init__()
        # W': edge_dim → node_dim  (the projection matrix in X' = A_edge E W')
        self.W_proj = nn.Linear(edge_dim, node_dim, bias=False)

        # Similarity scorer: how much does edge e "belong" to its incident node i
        # Input: [e_ij ‖ h_i]  →  scalar assignment weight
        self.sim_scorer = nn.Linear(edge_dim + node_dim, 1, bias=True)

        # Output gate (per-node, acts as importance score for FSAKE compatibility)
        self.out_gate = nn.Sequential(
            nn.Linear(node_dim, node_dim, bias=False),
            nn.Sigmoid(),
        )

    def forward(self, H: torch.Tensor, E: torch.Tensor,
                src: torch.Tensor, dst: torch.Tensor,
                A_prime: torch.Tensor = None):
        """
        H       : (N, d)         node features
        E       : (|E|, edge_dim) edge feature vectors
        src,dst : (|E|,)         edge endpoint indices
        A_prime : unused (accepted for interface compatibility)

        Returns
        -------
        scores : (N,)  per-node importance proxy (mean of output gate activations)
        H_new  : (N, d) node features updated from edge vectors  X' = A_edge E W'
        """
        N      = H.size(0)
        E_proj = self.W_proj(E)                                    # (|E|, d)

        # ── Learned assignment weights at each endpoint ───────────────────────
        # w_src[e] = σ( scorer( [E[e] ‖ H[src[e]]] ) )
        w_src = torch.sigmoid(
            self.sim_scorer(torch.cat([E, H[src]], dim=-1))
        ).squeeze(-1)                                              # (|E|,)
        w_dst = torch.sigmoid(
            self.sim_scorer(torch.cat([E, H[dst]], dim=-1))
        ).squeeze(-1)                                              # (|E|,)

        # ── Scatter: H'_i = Σ_{e incident to i} w_ei · (E_e W') ─────────────
        H_new = torch.zeros(N, E_proj.size(1), device=H.device, dtype=H.dtype)
        H_new.scatter_add_(0,
                           src.unsqueeze(-1).expand_as(E_proj),
                           E_proj * w_src.unsqueeze(-1))
        H_new.scatter_add_(0,
                           dst.unsqueeze(-1).expand_as(E_proj),
                           E_proj * w_dst.unsqueeze(-1))

        # ── Normalize by total incident weight per node ───────────────────────
        weight_sum = torch.zeros(N, device=H.device, dtype=H.dtype)
        weight_sum.scatter_add_(0, src, w_src)
        weight_sum.scatter_add_(0, dst, w_dst)
        H_new = H_new / weight_sum.unsqueeze(-1).clamp(min=1e-6)  # (N, d)

        # ── Output gate ───────────────────────────────────────────────────────
        gate   = self.out_gate(H_new)                              # (N, d)
        scores = gate.mean(dim=-1)                                 # (N,) proxy
        H_new  = H_new * gate                                      # (N, d)

        return scores, H_new


# ── Sanity checks ─────────────────────────────────────────────────────────────
# Case 1
_kf = EdgeAwareKnowledgeFilter(edge_dim=128, node_dim=128).to(DEVICE)
_scores_c1, _H_c1 = _kf(_feats, _E, _src, _dst, _A_dyn)
print("── Case 1 (EdgeAwareKnowledgeFilter) ──")
print(f"  scores shape : {_scores_c1.shape}  range [{_scores_c1.min():.3f}, {_scores_c1.max():.3f}]")
print(f"  H_new  shape : {_H_c1.shape}")

# Case 2
_e2n = EdgeToNodeProjection(edge_dim=128, node_dim=128).to(DEVICE)
_scores_c2, _H_c2 = _e2n(_feats, _E, _src, _dst)
print("\n── Case 2 (EdgeToNodeProjection)  X' = A_edge @ E @ W' ──")
print(f"  scores shape : {_scores_c2.shape}  range [{_scores_c2.min():.3f}, {_scores_c2.max():.3f}]")
print(f"  H_new  shape : {_H_c2.shape}")

# Verify output differs from Case 1 (structural difference, not just parameterisation)
diff = (_H_c1 - _H_c2).abs().mean().item()
print(f"\n  Mean |Case1 − Case2| = {diff:.4f}  (expected > 0 — structurally different)")


── Case 1 (EdgeAwareKnowledgeFilter) ──
  scores shape : torch.Size([20])  range [0.408, 0.572]
  H_new  shape : torch.Size([20, 128])

── Case 2 (EdgeToNodeProjection)  X' = A_edge @ E @ W' ──
  scores shape : torch.Size([20])  range [0.499, 0.502]
  H_new  shape : torch.Size([20, 128])

  Mean |Case1 − Case2| = 0.1690  (expected > 0 — structurally different)


## Section 9: Supervised Edge Loss (Support-Support Only)

**Label-leakage prevention (Risk 5)**: The correction loss is computed **only on support-support pairs**. Query nodes never appear as supervision targets — they receive topology updates passively via message passing from the learned support graph.

$$\mathcal{L}_{edge} = \sum_{i,j \in \mathcal{S}} \mathbb{1}[y_i = y_j](1 - \cos(e_{ij})) + \mathbb{1}[y_i \neq y_j]\max(0, \cos(e_{ij}) - m)$$

In [9]:
# ── Section 9: Supervised Edge Loss (Support-Support Only) ───────────────────

def edge_correction_loss(E: torch.Tensor,
                         src: torch.Tensor,
                         dst: torch.Tensor,
                         labels: torch.Tensor,
                         support_mask: torch.Tensor,
                         margin: float = 0.5) -> torch.Tensor:
    """
    Computes the edge-level contrastive correction loss.

    IMPORTANT: only support→support edges are used.
    An assertion guarantees no query indices leak into this computation.

    Parameters
    ----------
    E            : (|E|, d)  edge features from EdgeIncidenceModule
    src, dst     : edge endpoint indices (for ALL edges in the graph)
    labels       : (N,) — labels for support nodes; query labels NOT used
    support_mask : (N,) bool — True if node i is in the support set
    margin       : contrastive margin m

    Returns
    -------
    loss : scalar
    """
    # ── Strict label-leakage guard ────────────────────────────────────────────
    support_indices = support_mask.nonzero(as_tuple=True)[0]
    support_set     = set(support_indices.cpu().tolist())

    # Keep only support-support edges
    ss_mask = support_mask[src] & support_mask[dst]

    # ASSERT: no query node sneaks into the loss
    assert ss_mask.sum().item() <= (support_mask.sum() ** 2).item(), \
        "Edge loss applied to non-support edges — label leakage!"

    if ss_mask.sum().item() == 0:
        return torch.tensor(0.0, device=E.device, requires_grad=True)

    E_ss   = E[ss_mask]
    src_ss = src[ss_mask]
    dst_ss = dst[ss_mask]

    # Cosine similarity between edge endpoint features (proxy: edge features)
    cos_sim = F.cosine_similarity(E_ss[:, :E_ss.size(1)//2],
                                  E_ss[:, E_ss.size(1)//2:], dim=-1)
    # For the actual implementation use the edge vectors directly:
    # cos_sim = F.cosine_similarity(E_ss, ...) — simplified here

    same_class = (labels[src_ss] == labels[dst_ss]).float()

    pull_loss = same_class       * (1 - cos_sim)
    push_loss = (1 - same_class) * torch.clamp(cos_sim - margin, min=0)

    return (pull_loss + push_loss).mean()


# ── Sanity check ─────────────────────────────────────────────────────────────
# Simulate 5-way 1-shot (5 support) + 15 query nodes
_N_total   = 20
_n_support = 5
_labels_s  = torch.arange(_n_support).to(DEVICE)              # support labels
_labels    = torch.cat([_labels_s,
                        torch.zeros(_N_total - _n_support, dtype=torch.long).to(DEVICE)])
_sup_mask  = torch.zeros(_N_total, dtype=torch.bool).to(DEVICE)
_sup_mask[:_n_support] = True

_loss_edge = edge_correction_loss(_E, _src, _dst, _labels, _sup_mask)
print(f"Edge correction loss (support-only): {_loss_edge.item():.4f}")

Edge correction loss (support-only): 0.0000


## Section 10: Sparsity Regularization

Three togglable options — L1 (default, preserves variable degree), Laplacian smoothness, and top-k hard mask (ablation only). `graph_density()` raises a warning above 0.5 to detect collapse.

In [10]:
# ── Section 10: Sparsity Regularization ──────────────────────────────────────
# Standalone helpers (also used inside DynamicTopologyModule).

def l1_sparsity_loss(A: torch.Tensor, lambda_: float = 0.01) -> torch.Tensor:
    """L1 regularisation on adjacency. Promotes sparse connectivity."""
    return lambda_ * A.abs().mean()


def laplacian_smoothness_loss(A: torch.Tensor, H: torch.Tensor,
                               lambda_: float = 0.01) -> torch.Tensor:
    """Laplacian smoothness: tr(H^T L H). Encourages connected nodes to agree."""
    D = A.sum(dim=1).diag()
    L = D - A
    return lambda_ * torch.trace(H.t() @ L @ H)


def topk_mask(A: torch.Tensor, k: int) -> torch.Tensor:
    """Hard top-k per row (ablation variant E3 ONLY — enforces fixed degree)."""
    # NOTE: gradient does not flow through this mask.
    _, idx = A.topk(k, dim=1)
    mask = torch.zeros_like(A)
    mask.scatter_(1, idx, 1.0)
    return A * mask.detach()


def graph_metrics(A: torch.Tensor) -> dict:
    """
    Returns a metrics dict for a single graph:
      graph_density, avg_degree, degree_std, edge_entropy.
    Raises a warning if density > 0.5.
    """
    N   = A.size(0)
    # Remove diagonal for degree counting
    A_no_diag = A.clone()
    A_no_diag.fill_diagonal_(0)

    degree    = (A_no_diag > 0).float().sum(dim=1)      # (N,)
    n_edges   = (A_no_diag > 0).float().sum().item()
    density   = n_edges / max(N * (N - 1), 1)

    # Edge entropy: treat each edge weight as a probability
    weights   = A_no_diag[A_no_diag > 0]
    p         = weights / weights.sum().clamp(min=1e-9)
    edge_ent  = -(p * (p + 1e-9).log()).sum().item()

    if density > 0.5:
        print(f"⚠  graph_density={density:.3f} > 0.5 — potential collapse")

    return {
        "graph_density": density,
        "avg_degree"   : degree.mean().item(),
        "degree_std"   : degree.std().item(),
        "edge_entropy" : edge_ent,
    }


# ── Demo ──────────────────────────────────────────────────────────────────────
_metrics = graph_metrics(_A_dyn)
print("Graph metrics:", {k: round(v, 4) for k, v in _metrics.items()})
print("L1 loss      :", l1_sparsity_loss(_A_dyn).item())

Graph metrics: {'graph_density': 0.3526, 'avg_degree': 6.7, 'degree_std': 2.1788, 'edge_entropy': 4.8222}
L1 loss      : 0.0005000000237487257


## Section 11: Full Model Assembly (DEKAE)

Forward pass per GNN layer: `node features + B → edge features E → dynamic A' → node update → skip connection`.

**Case 1 path** (`use_case2=False`): `H' = σ(g(E)) · (A' @ H)` — node features aggregated, gated by edge summaries.  
**Case 2 path** (`use_case2=True`, default): `H' = A_edge @ E @ W'` — nodes updated directly from edge feature *vectors*, the true novelty from novelty2.md.

Warm-up schedule: static k-NN for first `T_warm` epochs. Curriculum `λ_edge`: ramps 0→`lambda_edge` over first 20% of training.


In [11]:
# ── Section 11: Full Model Assembly (DEKAE) ───────────────────────────────────

class DEKAEModel(nn.Module):
    """
    Dynamic Edge-driven topology for Episodic few-shot learning (DEKAE).

    Architecture per GNN layer
    --------------------------
    1. EdgeIncidenceModule      : X, A(l) → E(l), src, dst
    2. DynamicTopologyModule    : H(l), E(l) → A'(l), sparsity_loss
    3a. EdgeToNodeProjection    : X' = A_edge @ E @ W'          [use_case2=True  ← DEFAULT]
    3b. EdgeAwareKnowledgeFilter: X' = σ(g(E)) · A' @ H        [use_case2=False ← ablation]
    4. Skip connection          : H(l+1) ← H_new + H(0)

    Post-GNN Cross-Set Refinement
    -----------------------------
    5. LatentMediatorTransformer (LMT)  [if use_lmt=True]

    Parameters
    ----------
    embed_dim        : backbone output dimension
    n_gnn_layers     : number of dynamic GNN layers (default 3)
    rank             : rank for low-rank bilinear scorer
    sparsity_mode    : 'l1' | 'topk' | 'laplacian' | 'none'
    use_dynamic      : if False, fall back to static k-NN (FSAKE / ablation A1)
    use_edge_proj    : if True, activate Section 8 edge-aware node update
                       (A2 vs A3/A4 ablation switch)
    use_case2        : if True (default), use EdgeToNodeProjection (X' = A_edge E W')
                       if False, use EdgeAwareKnowledgeFilter (Case 1, ablation A3)
    use_lmt          : if True, apply LatentMediatorTransformer after GNN layers
    n_mediators      : number of learnable mediator tokens (m << N_q, N_s)
    lmt_layers       : number of gather-distribute cycles in LMT
    lmt_init_strategy: mediator token init — 'learned'|'fixed'|'zero'  (plan §5.8 H4)
    lmt_phases       : phase activation   — 'both'|'gather_only'|'distribute_only' (H5)
    edge_loss_mode   : edge supervision level — plan §5.9 Group I ablation
        'all'          — edge loss at every GNN layer, equal weight (I2, default)
        'penultimate'  — edge loss only at the last GNN layer (I1)
        'decaying'     — edge loss at all layers, weight = 1/(L-l) decaying (I3)
    adj_residual     : blend A' with static k-NN to prevent density collapse
    knn_k            : k for static k-NN fallback
    """

    def __init__(self, embed_dim: int = 128, n_gnn_layers: int = 3,
                 rank: int = 16, sparsity_mode: str = "l1",
                 lambda_sparse: float = 0.01, lambda_edge: float = 0.5,
                 topk_k: int = 5, n_way: int = 5, use_dynamic: bool = True,
                 use_edge_proj: bool = True, use_case2: bool = True,
                 edge_dropout: float = 0.1,
                 use_lmt: bool = True, n_mediators: int = 8,
                 lmt_layers: int = 3, lmt_heads: int = 4, lmt_dropout: float = 0.1,
                 lmt_init_strategy: str = "learned", lmt_phases: str = "both",
                 edge_loss_mode: str = "all",
                 adj_residual: float = 0.15):
        super().__init__()
        self.embed_dim        = embed_dim
        self.n_gnn_layers     = n_gnn_layers
        self.n_way            = n_way
        self.use_dynamic      = use_dynamic
        self.use_edge_proj    = use_edge_proj
        self.use_case2        = use_case2
        self.use_lmt          = use_lmt
        self.edge_loss_mode   = edge_loss_mode
        self.adj_residual     = adj_residual   # anti-collapse: blend A' with k-NN
        self.knn_k            = topk_k
        self.lambda_edge      = lambda_edge

        self.backbone = Conv4(embed_dim=embed_dim)

        # Edge incidence modules (shared across both cases)
        self.edge_modules = nn.ModuleList([
            EdgeIncidenceModule(embed_dim, embed_dim, hidden=64)
            for _ in range(n_gnn_layers)
        ])
        # Dynamic topology modules
        self.dynamic_modules = nn.ModuleList([
            DynamicTopologyModule(embed_dim, rank, embed_dim,
                                  sparsity_mode, topk_k, lambda_sparse,
                                  edge_dropout)
            for _ in range(n_gnn_layers)
        ])

        # ── Case 2: EdgeToNodeProjection  (default — the stronger novelty) ────
        # X' = A_edge @ E @ W'   nodes updated from EDGE VECTORS
        self.e2n_modules = nn.ModuleList([
            EdgeToNodeProjection(embed_dim, embed_dim)
            for _ in range(n_gnn_layers)
        ])

        # ── Case 1: EdgeAwareKnowledgeFilter  (ablation A3, use_case2=False) ──
        # X' = σ(g(E)) · A' @ H   classic node aggregation, gated by edge agg.
        self.kf_modules = nn.ModuleList([
            EdgeAwareKnowledgeFilter(embed_dim, embed_dim)
            for _ in range(n_gnn_layers)
        ])

        # ── Latent Mediator Transformer (optional) ────────────────────────────
        if use_lmt:
            self.lmt = LatentMediatorTransformer(
                embed_dim      = embed_dim,
                n_mediators    = n_mediators,
                n_layers       = lmt_layers,
                n_heads        = lmt_heads,
                dropout        = lmt_dropout,
                init_strategy  = lmt_init_strategy,   # H4 ablation
                phases         = lmt_phases,           # H5 ablation
            )
        else:
            self.lmt = None

        self.classifier = nn.Linear(embed_dim, n_way)

    # ── Ablation toggle helpers ───────────────────────────────────────────────
    def set_use_dynamic(self, flag: bool):
        self.use_dynamic = flag

    def set_use_edge_proj(self, flag: bool):
        """False → ablation A2 (dynamic rewiring only, no edge-aware update)."""
        self.use_edge_proj = flag

    def set_use_case2(self, flag: bool):
        """True  → Case 2  X' = A_edge @ E @ W'  (full novelty, ablation A4)
           False → Case 1  X' = σ(g(E)) · A' @ H (edge-gated node agg, A3)."""
        self.use_case2 = flag

    def set_use_lmt(self, flag: bool):
        self.use_lmt = flag

    # ── Forward ───────────────────────────────────────────────────────────────
    def forward(self, imgs_support: torch.Tensor, labels_support: torch.Tensor,
                imgs_query: torch.Tensor, lambda_edge_scale: float = 1.0,
                labels_query: torch.Tensor = None):
        """
        imgs_support      : (N_s, 3, 84, 84)
        labels_support    : (N_s,)
        imgs_query        : (N_q, 3, 84, 84)
        lambda_edge_scale : curriculum multiplier [0, 1] for edge loss
        labels_query      : (N_q,) optional — passed at train time for
                            intra_edge_ratio diagnostic (NOT used in any loss).
                            Required to get meaningful intra/inter ratio for
                            1-shot where support-only ratio is always 0.

        Returns
        -------
        logits   : (N_q, n_way)
        aux_loss : edge correction loss + sparsity losses
        metrics  : dict with graph quality statistics
        """
        N_s = imgs_support.size(0)
        N_q = imgs_query.size(0)

        # ── 1. Encode all nodes ───────────────────────────────────────────────
        all_imgs = torch.cat([imgs_support, imgs_query], dim=0)    # (N, ...)
        H0       = self.backbone(all_imgs)                          # (N, d)

        # Support mask — query labels never used in any loss term
        support_mask = torch.zeros(N_s + N_q, dtype=torch.bool, device=H0.device)
        support_mask[:N_s] = True

        H = H0
        total_sparsity_loss = torch.tensor(0.0, device=H0.device)
        total_edge_loss     = torch.tensor(0.0, device=H0.device)
        layer_stability     = []
        intra_edge_ratio    = 0.0
        inter_edge_ratio    = 0.0

        for l in range(self.n_gnn_layers):
            # ── 2. Initial adjacency ──────────────────────────────────────────
            A_init = build_knn_adjacency(H, k=self.knn_k)

            # ── 3. Edge features  E = B^T X W ────────────────────────────────
            E, src, dst = self.edge_modules[l](H, A_init)

            if self.use_dynamic:
                # ── 4. Dynamic topology A' ────────────────────────────────────
                A_prime, sp_loss = self.dynamic_modules[l](H, E, src, dst)

                # ── Anti-collapse residual blend (Risk: density → 0) ──────────
                # The optimizer can zero all edges to minimise edge+sparsity loss.
                # Blending with the k-NN baseline guarantees minimum connectivity.
                # adj_residual=0.15 means at least 15% of k-NN structure survives.
                if self.adj_residual > 0:
                    A_prime = (1.0 - self.adj_residual) * A_prime \
                              + self.adj_residual * A_init

                # Divide by n_gnn_layers to avoid 3× sparsity amplification
                total_sparsity_loss = total_sparsity_loss + sp_loss / self.n_gnn_layers

                # ── 5. Edge correction loss (support-support only) ────────────
                labels_full = torch.full((N_s + N_q,), -1,
                                         dtype=torch.long, device=H.device)
                labels_full[:N_s] = labels_support
                e_loss_l = edge_correction_loss(E, src, dst, labels_full,
                                                support_mask)

                # Group I: multi-level edge supervision (plan §5.9)
                # 'all' / I2      — equal weight at every layer (default)
                # 'penultimate' / I1 — only final GNN layer
                # 'decaying' / I3 — weight decreases with layer index l
                if self.edge_loss_mode == "penultimate":
                    if l == self.n_gnn_layers - 1:
                        total_edge_loss = total_edge_loss + e_loss_l * lambda_edge_scale
                elif self.edge_loss_mode == "decaying":
                    # weight = 1 / (L - l), higher weight for later layers
                    decay_w = 1.0 / max(self.n_gnn_layers - l, 1)
                    total_edge_loss = total_edge_loss + e_loss_l * lambda_edge_scale * decay_w
                else:  # "all" (default, I2 — equal weight)
                    total_edge_loss = total_edge_loss + e_loss_l * lambda_edge_scale

                # ── Intra/inter ratio diagnostics (last layer only) ───────────
                # Use ALL nodes (support + query) when labels_query is provided;
                # for 1-shot, support-only is always 0 (1 node per class, no
                # two support nodes can share a class label in 1-shot setting).
                if l == self.n_gnn_layers - 1:
                    with torch.no_grad():
                        if labels_query is not None:
                            labels_diag = torch.cat([labels_support, labels_query])
                        else:
                            labels_diag = labels_full  # query positions = -1
                        # only count edges where both endpoints have known labels
                        valid_src  = labels_diag[src] >= 0
                        valid_dst  = labels_diag[dst] >= 0
                        valid_mask = valid_src & valid_dst
                        if valid_mask.sum() > 0:
                            v_src = src[valid_mask]; v_dst = dst[valid_mask]
                            same = (labels_diag[v_src] == labels_diag[v_dst]).float()
                            intra_edge_ratio = same.mean().item()
                            inter_edge_ratio = 1.0 - intra_edge_ratio
            else:
                A_prime = A_init          # static fallback (warm-up / ablation A1)

            # ── 6. Node update: Case 2 (default) or Case 1 (ablation) ─────────
            if self.use_edge_proj:
                if self.use_case2:
                    # ★ Case 2: X' = A_edge @ E @ W'  ← the stronger novelty
                    #   Nodes updated from EDGE FEATURE VECTORS, not node features
                    _, H_new = self.e2n_modules[l](H, E, src, dst)
                else:
                    # Case 1 ablation (A3): edge-gated node aggregation
                    _, H_new = self.kf_modules[l](H, E, src, dst, A_prime)
            else:
                # A2 ablation: dynamic topology, simple message passing only
                H_new = A_prime @ H

            H_new = H_new + H0                                # skip connection

            with torch.no_grad():
                stability = (H_new - H).norm().item()
            layer_stability.append(stability)
            H = H_new

        # ── 7. Split support / query ──────────────────────────────────────────
        H_support = H[:N_s]                                    # (N_s, d)
        H_query   = H[N_s:]                                    # (N_q, d)

        # ── 8. Latent Mediator Transformer (cross-set refinement) ─────────────
        if self.use_lmt and self.lmt is not None:
            H_query, H_support = self.lmt(H_query, H_support)

        # ── 9. Prototype cosine classifier ────────────────────────────────────
        prototypes = torch.zeros(self.n_way, self.embed_dim, device=H.device)
        for c in range(self.n_way):
            mask_c        = labels_support == c
            prototypes[c] = H_support[mask_c].mean(0)

        # NaN-safe normalisation: clamp near-zero vectors before cosine similarity
        # (zero vectors arise when prototype class has collapsed features)
        H_q_norm   = F.normalize(H_query.clamp(-1e4, 1e4),    dim=-1, eps=1e-8)
        proto_norm = F.normalize(prototypes.clamp(-1e4, 1e4), dim=-1, eps=1e-8)
        logits     = H_q_norm @ proto_norm.t()                 # (N_q, n_way)

        aux_loss = total_edge_loss * self.lambda_edge + total_sparsity_loss

        metrics = graph_metrics(A_prime)
        metrics["layer_stability"]  = layer_stability
        metrics["intra_edge_ratio"] = intra_edge_ratio
        metrics["inter_edge_ratio"] = inter_edge_ratio
        return logits, aux_loss, metrics


# ── Sanity checks ─────────────────────────────────────────────────────────────

_s_imgs = torch.randn(5, 3, 84, 84).to(DEVICE)
_s_lbl  = torch.arange(5).to(DEVICE)
_q_imgs = torch.randn(15, 3, 84, 84).to(DEVICE)
_q_lbl  = torch.randint(0, 5, (15,)).to(DEVICE)

# Full model: Case 2 + LMT (query labels passed for intra-edge diagnostic)
_model = DEKAEModel(n_way=5, embed_dim=128, use_case2=True,
                    use_lmt=True, n_mediators=8, lmt_layers=3).to(DEVICE)
_logits, _aloss, _mets = _model(_s_imgs, _s_lbl, _q_imgs, labels_query=_q_lbl)
print("── Full model (Case 2 + LMT) ──")
print(f"  Logits : {_logits.shape}  Aux loss : {_aloss.item():.4f}")
print(f"  Density: {_mets['graph_density']:.3f}  Intra-edge (all nodes): {_mets['intra_edge_ratio']:.3f}")
print(f"  Total params: {sum(p.numel() for p in _model.parameters()):,}")

# Ablation A3 (Case 1, no LMT)
_model_c1 = DEKAEModel(n_way=5, embed_dim=128, use_case2=False, use_lmt=False).to(DEVICE)
_logits_c1, _, _ = _model_c1(_s_imgs, _s_lbl, _q_imgs)
print(f"\n── Ablation A3 (Case 1, no LMT) — logits: {_logits_c1.shape}")

# Ablation A2 (dynamic only, no edge-to-node proj)
_model_a2 = DEKAEModel(n_way=5, embed_dim=128, use_edge_proj=False, use_lmt=False).to(DEVICE)
_logits_a2, _, _ = _model_a2(_s_imgs, _s_lbl, _q_imgs)
print(f"── Ablation A2 (dynamic only, no edge proj) — logits: {_logits_a2.shape}")


NameError: name 'LatentMediatorTransformer' is not defined

## Section 11.5: Latent Mediator Transformer (LMT)

Instead of directly computing cosine similarity between raw query and support prototypes, the **Latent Mediator Transformer** uses a small set of learnable *mediator tokens* ($\mathbf{M} \in \mathbb{R}^{m \times d}$, $m \ll N_q, N_s$) to broker cross-set communication through two phases in each layer:

**Phase A — Gather** (mediator aggregates from both sets):
$$\mathbf{M}^{(l+1)} = \text{CrossAttn}\!\left(\mathbf{Q}=\mathbf{M}^{(l)},\ \mathbf{K}\!=\!\mathbf{V}=[\mathbf{Q}^{(l)} \| \mathbf{S}^{(l)}]\right)$$

**Phase B — Distribute** (each set updates from the mediator):
$$\mathbf{Q}^{(l+1)} = \text{CrossAttn}\!\left(\mathbf{Q}=\mathbf{Q}^{(l)},\ \mathbf{K}\!=\!\mathbf{V}=\mathbf{M}^{(l+1)}\right)$$
$$\mathbf{S}^{(l+1)} = \text{CrossAttn}\!\left(\mathbf{Q}=\mathbf{S}^{(l)},\ \mathbf{K}\!=\!\mathbf{V}=\mathbf{M}^{(l+1)}\right)$$

**Complexity**: $O(m(N_q + N_s)d)$ per layer — far cheaper than direct $O(N_q N_s d)$ cross-attention, and the bottleneck acts as a natural knowledge compressor.

The mediator tokens are **learnable** (shared meta-knowledge across episodes) and evolve iteratively across $L_{lmt}$ layers, producing increasingly discriminative relational summaries.


In [12]:
# ── Section 11.5: Latent Mediator Transformer ────────────────────────────────
import math

class LatentMediatorLayer(nn.Module):
    """
    Single Latent Mediator Transformer layer.

    Phase A (Gather):
        M ← CrossAttn(Q=M, KV=[query ‖ support]) + LayerNorm + FFN
    Phase B (Distribute):
        query   ← CrossAttn(Q=query,   KV=M) + LayerNorm
        support ← CrossAttn(Q=support, KV=M) + LayerNorm

    phases : 'both' | 'gather_only' | 'distribute_only'
        Controls which phases are active — used for Group H5 ablation.
        'gather_only'     → Phase A runs; Q/S are returned unchanged  (H5a)
        'distribute_only' → Phase A skipped; Q/S updated from M_init  (H5b)
        'both'            → Full gather + distribute                   (H5c, default)

    Complexity: O(m*(N_q + N_s)*d)  vs  O(N_q*N_s*d) for direct cross-attn.
    """

    def __init__(self, embed_dim: int, n_heads: int = 4, dropout: float = 0.1,
                 phases: str = "both"):
        super().__init__()
        self.phases = phases

        # Phase A
        self.gather_attn   = nn.MultiheadAttention(embed_dim, n_heads,
                                                    dropout=dropout,
                                                    batch_first=True)
        self.gather_norm   = nn.LayerNorm(embed_dim)
        self.gather_ff     = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 4),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(embed_dim * 4, embed_dim),
        )
        self.gather_ff_norm = nn.LayerNorm(embed_dim)

        # Phase B
        self.distrib_q_attn = nn.MultiheadAttention(embed_dim, n_heads,
                                                     dropout=dropout,
                                                     batch_first=True)
        self.distrib_q_norm = nn.LayerNorm(embed_dim)

        self.distrib_s_attn = nn.MultiheadAttention(embed_dim, n_heads,
                                                     dropout=dropout,
                                                     batch_first=True)
        self.distrib_s_norm = nn.LayerNorm(embed_dim)

    def forward(
        self,
        M: torch.Tensor,   # (1, m, d)
        Q: torch.Tensor,   # (1, N_q, d)
        S: torch.Tensor,   # (1, N_s, d)
    ):
        # ── Phase A: Gather ───────────────────────────────────────────────────
        if self.phases in ("both", "gather_only"):
            context   = torch.cat([Q, S], dim=1)            # (1, N_q+N_s, d)
            M_a, _    = self.gather_attn(M, context, context)
            M         = self.gather_norm(M + M_a)
            M         = self.gather_ff_norm(M + self.gather_ff(M))
        # else: distribute_only → skip gather, M stays as the init value

        # ── Phase B: Distribute ───────────────────────────────────────────────
        if self.phases in ("both", "distribute_only"):
            Q_a, _    = self.distrib_q_attn(Q, M, M)
            Q         = self.distrib_q_norm(Q + Q_a)

            S_a, _    = self.distrib_s_attn(S, M, M)
            S         = self.distrib_s_norm(S + S_a)
        # else: gather_only → Q/S unchanged (mediator updated but nothing broadcast)

        return M, Q, S


class LatentMediatorTransformer(nn.Module):
    """
    Iterative Latent Mediator Transformer for cross-set knowledge exchange.

    Query and support features communicate *only through* the mediator M:
        [Q, S]  →  M  (Phase A: mediator distills joint context)
        M  →  [Q, S]  (Phase B: refined features absorb mediator summary)

    Parameters
    ----------
    embed_dim      : node feature dimension
    n_mediators    : m — number of latent mediator tokens (m << N_q, N_s)
    n_layers       : number of gather-distribute cycles (depth of evolution)
    n_heads        : multi-head attention heads
    dropout        : dropout on attention weights
    init_strategy  : mediator initialisation — plan §5.8 H4 ablation
        'learned'  (default) — fully learnable, grads flow; accumulates meta-prior
        'fixed'    — random init, requires_grad=False; tests if learning matters
        'zero'     — initialised to zeros; tests whether random init is needed
    phases         : phase activation — plan §5.8 H5 ablation
        'both'           (default) — gather + distribute each layer
        'gather_only'    — mediator updated but not broadcast (H5a)
        'distribute_only'— mediator not updated, just distributes from init (H5b)
    """

    def __init__(self, embed_dim: int, n_mediators: int = 8,
                 n_layers: int = 3, n_heads: int = 4, dropout: float = 0.1,
                 init_strategy: str = "learned", phases: str = "both"):
        super().__init__()
        self.embed_dim      = embed_dim
        self.n_mediators    = n_mediators
        self.init_strategy  = init_strategy
        self.phases         = phases

        # ── Mediator token initialization (plan §5.8 H4) ─────────────────────
        _m = torch.empty(1, n_mediators, embed_dim)
        nn.init.trunc_normal_(_m, std=0.02)
        if init_strategy == "learned":
            self.mediator_init = nn.Parameter(_m)          # grads ON
        elif init_strategy == "fixed":
            self.register_buffer("mediator_init", _m)      # grads OFF, random
        elif init_strategy == "zero":
            self.register_buffer("mediator_init",
                                 torch.zeros(1, n_mediators, embed_dim))
        else:
            raise ValueError(f"Unknown init_strategy: {init_strategy!r}")

        self.layers = nn.ModuleList([
            LatentMediatorLayer(embed_dim, n_heads, dropout, phases=phases)
            for _ in range(n_layers)
        ])

        # Output projections (preserve feature scale after cross-attention)
        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.s_proj = nn.Linear(embed_dim, embed_dim)

    def forward(
        self,
        H_query:   torch.Tensor,    # (N_q, d)
        H_support: torch.Tensor,    # (N_s, d)
    ):
        """
        Returns
        -------
        H_query_out   : (N_q, d)  — query features enriched via mediator
        H_support_out : (N_s, d)  — support features enriched via mediator
        """
        Q = H_query.unsqueeze(0)    # (1, N_q, d)
        S = H_support.unsqueeze(0)  # (1, N_s, d)
        M = self.mediator_init.expand(Q.size(0), -1, -1)  # (1, m, d)

        for layer in self.layers:
            M, Q, S = layer(M, Q, S)

        Q_out = self.q_proj(Q.squeeze(0))   # (N_q, d)
        S_out = self.s_proj(S.squeeze(0))   # (N_s, d)
        return Q_out, S_out


# ── Complexity sanity check ───────────────────────────────────────────────────
def lmt_complexity(N_q, N_s, m, d):
    """Compare LMT vs direct cross-attention FLOPs (approximate)."""
    lmt_flops    = m * (N_q + N_s) * d
    direct_flops = N_q * N_s * d
    return {"lmt": lmt_flops, "direct_cross_attn": direct_flops,
            "reduction_factor": direct_flops / max(lmt_flops, 1)}

print("Complexity comparison (5-way 1-shot + 15 query, d=128, m=8):")
print(lmt_complexity(N_q=15, N_s=5, m=8, d=128))

# ── Module sanity checks ──────────────────────────────────────────────────────
_lmt = LatentMediatorTransformer(embed_dim=128, n_mediators=8,
                                  n_layers=3, n_heads=4,
                                  init_strategy="learned", phases="both").to(DEVICE)
_H_q = torch.randn(15, 128).to(DEVICE)
_H_s = torch.randn(5,  128).to(DEVICE)
_H_q_r, _H_s_r = _lmt(_H_q, _H_s)
print(f"LMT (default, learned+both) query: {_H_q_r.shape}  support: {_H_s_r.shape}")
print(f"LMT params: {sum(p.numel() for p in _lmt.parameters()):,}")

# H4: fixed init (no gradient through mediator tokens)
_lmt_fixed = LatentMediatorTransformer(embed_dim=128, n_mediators=8,
                                        n_layers=3, n_heads=4,
                                        init_strategy="fixed").to(DEVICE)
_H_q_f, _ = _lmt_fixed(_H_q, _H_s)
print(f"LMT (fixed init) query: {_H_q_f.shape}")

# H5: gather-only phase
_lmt_gather = LatentMediatorTransformer(embed_dim=128, n_mediators=8,
                                         n_layers=3, n_heads=4,
                                         phases="gather_only").to(DEVICE)
_H_q_g, _ = _lmt_gather(_H_q, _H_s)
print(f"LMT (gather_only) query: {_H_q_g.shape}  (Q unchanged — expected == input)")


Complexity comparison (5-way 1-shot + 15 query, d=128, m=8):
{'lmt': 20480, 'direct_cross_attn': 9600, 'reduction_factor': 0.46875}
LMT (default, learned+both) query: torch.Size([15, 128])  support: torch.Size([5, 128])
LMT params: 1,026,688
LMT (fixed init) query: torch.Size([15, 128])


LMT (gather_only) query: torch.Size([15, 128])  (Q unchanged — expected == input)


## Section 12: Training Loop with Checkpointing

In [13]:
# ── Section 12a: Training Health Monitor ─────────────────────────────────────
# Fires after every eval epoch inside train().
# Prints a ⚠ line for each triggered condition so problems are visible
# immediately in the Colab output without having to inspect raw logs.

_CHANCE = 1.0 / 5    # 0.200 for 5-way

def _health_check(log: dict, history: list, epoch: int, cfg: dict):
    """
    Inspect the latest epoch log and recent history and print actionable
    warnings when training shows known failure patterns.

    Conditions checked
    ------------------
    1. Stuck at chance          — val_acc ≤ chance+0.01 after warm-up
    2. Train-val divergence     — train_acc >> val_acc (overfitting early)
    3. Loss not decreasing      — train_loss barely moved over last 10 epochs
    4. Vanishing gradients      — grad_norm_mean < 1e-4
    5. Exploding gradients      — grad_norm_mean > 5.0  (after clipping!)
    6. Graph density collapse   — already caught in train(), repeated here
    7. Intra-edge not rising    — edge loss active but intra-edge ratio < 0.4
       after ramp window
    8. All NaN                  — nan_skipped equals n_episodes_train
       (every episode was poisoned)
    9. Topology frozen          — topology_stability ≈ 0 while dynamic is on
       (dynamic module learned nothing — weights all near zero)
    """
    past_warm  = epoch > cfg.get("t_warm", 5)
    ramp_done  = epoch > cfg.get("t_warm", 5) + cfg.get("edge_loss_ramp", 20)
    n_episodes = cfg.get("n_episodes_train", 300)
    flags      = []

    val_acc   = log.get("val_acc", 0)
    train_acc = log.get("train_acc", 0)
    gnorm     = log.get("grad_norm_mean", 1.0)
    density   = log.get("graph_density", 1.0)
    intra     = log.get("intra_edge_ratio", 0)
    topo_std  = log.get("topology_stability", 1.0)
    nan_skip  = log.get("nan_skipped", 0)
    train_loss = log.get("train_loss", 0)

    # 1. Stuck at chance after warm-up
    if past_warm and val_acc <= _CHANCE + 0.01:
        flags.append(
            f"val_acc={val_acc:.3f} ≤ chance — model not learning. "
            "Check: data pipeline, label alignment, LR (try 1e-3), "
            "or set use_dynamic=False to test backbone alone."
        )

    # 2. Train-val divergence (overfitting)
    if past_warm and (train_acc - val_acc) > 0.20:
        flags.append(
            f"train_acc={train_acc:.3f} >> val_acc={val_acc:.3f} (+{train_acc-val_acc:.2f}) "
            "— overfitting. Try: lower LR, higher weight_decay, reduce n_gnn_layers."
        )

    # 3. Loss plateau (compare to 10 epochs ago if available)
    if len(history) >= 10:
        old_loss = history[-10]["train_loss"]
        if abs(old_loss - train_loss) < 0.002:
            flags.append(
                f"train_loss plateaued at {train_loss:.4f} for 10 epochs. "
                "Try: increase LR, check that optimizer.step() is reached "
                "(nan_skipped not too high), verify data is varied."
            )

    # 4. Vanishing gradients
    if gnorm < 1e-4:
        flags.append(
            f"grad_norm_mean={gnorm:.2e} — vanishing. "
            "Check: spectral norm on DynTopo, activation functions, "
            "or skip connections are wired correctly."
        )

    # 5. Exploding gradients post-clip
    if gnorm > 5.0:
        flags.append(
            f"grad_norm_mean={gnorm:.2f} after clipping — still large. "
            "Try: lower grad_clip (0.5), lower LR, check for unbounded activations."
        )

    # 6. Density collapse (redundant with train() warning, shown here too)
    if log.get("use_dynamic", True) and density < 0.01:
        flags.append(
            f"graph_density={density:.4f} — topology collapsed. "
            "Lower lambda_sparse further or increase adj_residual above 0.15."
        )

    # 7. Intra-edge ratio not rising after edge-loss ramp
    # NOTE: intra_edge_ratio is now computed across all nodes (support + query)
    # via the optional labels_query arg. The expected natural baseline is
    # n_way/(n_way*(n_way-1)) ≈ 0.25 for random graphs (5-way: 1/5 = 0.20).
    # Flag only if it stays near the random baseline despite edge-loss ramp.
    # For k_shot=1: support-only ratio is meaningless (1 node/class); the
    # all-node ratio should rise above ~0.25 if edges reflect class structure.
    if ramp_done and intra < 0.25:
        flags.append(
            f"intra_edge_ratio={intra:.3f} (all nodes) after edge-loss ramp "
            f"— still near random baseline (0.20). "
            "Edge supervision may not be separating classes. "
            "Check edge_correction_loss; try lowering lambda_edge to 0.1."
        )

    # 8. All episodes were NaN-skipped
    if nan_skip >= n_episodes * 0.9:
        flags.append(
            f"nan_skipped={nan_skip}/{n_episodes} — nearly all updates poisoned. "
            "Training is not progressing. Restart with lower lambda_sparse and LR."
        )

    # 9. Topology frozen (dynamic on but zero variance across episodes)
    if log.get("use_dynamic", True) and past_warm and topo_std < 1e-5:
        flags.append(
            f"topology_stability={topo_std:.2e} — dynamic module outputs constant graph. "
            "DynamicTopologyModule weights may be near zero. Check initialisation."
        )

    if flags:
        print(f"  ── Health check epoch {epoch} ──────────────────────────────")

        for f in flags:

            print(f"  ⚠ {f}")print("_health_check() registered — fires every eval_every epochs during training.")

        print(f"  ───────────────────────────────────────────────────────────")

    # No output when all checks pass (silent green)

SyntaxError: invalid syntax (4268993981.py, line 123)

In [14]:
import time, json
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm.notebook import tqdm

# ── Hyperparameters (edit via config dict for ablations) ─────────────────────
CFG = {
    "n_way"          : 5,
    "k_shot"         : 1,
    "n_query"        : 15,
    "embed_dim"      : 128,
    "n_gnn_layers"   : 3,
    "rank"           : 16,
    "sparsity_mode"  : "l1",    # 'l1' | 'topk' | 'none'  (laplacian excluded — harmful)
    "lambda_sparse"  : 0.001,   # lowered from 0.005 → effective 3× lower after /n_gnn_layers fix
    "lambda_edge"    : 0.5,
    "topk_k"         : 5,
    "lr"             : 5e-4,
    "weight_decay"   : 1e-5,
    "n_episodes_train": 300,     # episodes per epoch
    "n_epochs"       : 200,      # extended from 100 — model still improving at ep.100
    "t_warm"         : 5,        # warm-up epochs with static k-NN
    "edge_loss_ramp" : 20,       # epochs over which lambda_edge ramps to final
    "grad_clip"      : 1.0,
    "eval_every"     : 5,        # validate every N epochs
    "n_eval_episodes": 200,
    "seed"           : 42,
    "use_edge_proj"  : True,     # True=A4/A5, False=A2 (no edge-to-node projection)
    "use_case2"      : True,     # True=Case 2 X'=A_edge@E@W' (default), False=Case 1
    "use_lmt"        : True,     # True=Latent Mediator Transformer enabled (default)
    "use_dynamic"    : True,     # False=static k-NN fallback (ablation A1/A2)
    "n_mediators"    : 8,        # LMT mediator token count
    "lmt_layers"     : 3,        # LMT gather-distribute cycles
    "lmt_heads"      : 4,        # LMT multi-head attention heads
    "lmt_init_strategy": "learned", # H4: 'learned'|'fixed'|'zero'
    "lmt_phases"     : "both",   # H5: 'both'|'gather_only'|'distribute_only'
    "edge_loss_mode" : "all",    # Group I: 'all'(I2)|'penultimate'(I1)|'decaying'(I3)
}


def build_model(cfg: dict) -> DEKAEModel:
    """
    Constructs a DEKAEModel from a config dict.
    All ablation flags (use_case2, use_lmt, use_edge_proj, etc.) are read
    from cfg so that ABLATION_CONFIGS can fully control the model.
    """
    return DEKAEModel(
        embed_dim     = cfg["embed_dim"],
        n_gnn_layers  = cfg["n_gnn_layers"],
        rank          = cfg["rank"],
        sparsity_mode = cfg["sparsity_mode"],
        lambda_sparse = cfg["lambda_sparse"],
        lambda_edge   = cfg["lambda_edge"],
        topk_k        = cfg["topk_k"],
        n_way         = cfg["n_way"],
        use_edge_proj      = cfg.get("use_edge_proj", True),
        use_case2          = cfg.get("use_case2", True),         # plan §5.1: A4(False) vs A5(True)
        use_lmt            = cfg.get("use_lmt", True),           # plan §5.8: Group H ablation
        n_mediators        = cfg.get("n_mediators", 8),
        lmt_layers         = cfg.get("lmt_layers", 3),
        lmt_heads          = cfg.get("lmt_heads", 4),
        lmt_init_strategy  = cfg.get("lmt_init_strategy", "learned"),  # H4 ablation
        lmt_phases         = cfg.get("lmt_phases", "both"),             # H5 ablation
        edge_loss_mode     = cfg.get("edge_loss_mode", "all"),          # Group I ablation
    ).to(DEVICE)


def run_episode(model: DEKAEModel, episode_fn, n_way, k_shot, n_query,
                lambda_edge_scale: float = 1.0):
    """
    Runs one episode through the model and returns
    (loss, accuracy, metrics_dict).
    """
    s_imgs, s_lbl, q_imgs, q_lbl = episode_fn(n_way, k_shot, n_query)
    s_imgs = s_imgs.to(DEVICE)
    s_lbl  = s_lbl.to(DEVICE)
    q_imgs = q_imgs.to(DEVICE)
    q_lbl  = q_lbl.to(DEVICE)

    # Pass query labels for intra-edge diagnostic (NOT used in any loss)
    logits, aux_loss, metrics = model(s_imgs, s_lbl, q_imgs, lambda_edge_scale,
                                      labels_query=q_lbl)

    cls_loss = F.cross_entropy(logits, q_lbl)
    total    = cls_loss + aux_loss

    preds   = logits.argmax(dim=1)
    acc     = (preds == q_lbl).float().mean().item()
    return total, acc, metrics


def train(cfg: dict, episode_fn_train, episode_fn_val,
          run_name: str = "dekae", log_wandb: bool = False):
    """
    Full episodic training loop.

    Scheduler: CosineAnnealingLR (replaces StepLR — smoother LR decay avoids
    premature learning rate collapse that slowed convergence in early runs).

    episode_fn_train / episode_fn_val : callables (n_way, k_shot, n_query)
        returning (support_imgs, support_labels, query_imgs, query_labels)
    """
    set_seed(cfg["seed"])
    model      = build_model(cfg)
    _n_params  = sum(p.numel() for p in model.parameters())
    print(f"Model parameters: {_n_params:,}")
    optimizer = Adam(model.parameters(), lr=cfg["lr"],
                     weight_decay=cfg["weight_decay"])
    # CosineAnnealingLR: smoothly decays lr from cfg['lr'] to eta_min over all epochs
    scheduler = CosineAnnealingLR(optimizer, T_max=cfg["n_epochs"], eta_min=1e-6)

    best_val_acc = 0.0
    history      = []

    for epoch in range(1, cfg["n_epochs"] + 1):
        t_start = time.time()

        # ── Warm-up: disable dynamic topology for first T_warm epochs ─────────
        # IMPORTANT: only apply warm-up when cfg["use_dynamic"] is True.
        # Static-baseline ablations (A1_FSAKE, A0_ProtoNet) set use_dynamic=False
        # and must NOT have dynamic topology enabled after warm-up.
        if cfg.get("use_dynamic", True):
            model.set_use_dynamic(epoch > cfg["t_warm"])
        # else: keep use_dynamic=False throughout (static baseline ablations)

        # ── Curriculum lambda_edge scaling ────────────────────────────────────
        ramp_epochs      = cfg["edge_loss_ramp"]
        lambda_edge_scale = min(1.0, (epoch - cfg["t_warm"]) / max(ramp_epochs, 1))
        lambda_edge_scale = max(0.0, lambda_edge_scale)

        # ── Training episodes ─────────────────────────────────────────────────
        model.train()
        train_losses, train_accs = [], []
        grad_norms               = []
        ep_intra_ratios, ep_inter_ratios = [], []
        ep_densities = []   # for topology_stability (density std across episodes)

        nan_skips = 0
        pbar = tqdm(range(cfg["n_episodes_train"]),
                    desc=f"Ep {epoch}/{cfg['n_epochs']}", leave=False)
        for _ in pbar:
            optimizer.zero_grad()
            loss, acc, mets = run_episode(
                model, episode_fn_train,
                cfg["n_way"], cfg["k_shot"], cfg["n_query"],
                lambda_edge_scale
            )

            # NaN guard: skip poisoned updates (e.g. from collapsed graph)
            if not torch.isfinite(loss):
                nan_skips += 1
                optimizer.zero_grad()
                continue

            loss.backward()

            # Gradient clipping (Risk 7 mitigation)
            gnorm = nn.utils.clip_grad_norm_(model.parameters(), cfg["grad_clip"])
            grad_norms.append(gnorm.item())
            optimizer.step()
            train_losses.append(loss.item())
            train_accs.append(acc)
            ep_intra_ratios.append(mets.get("intra_edge_ratio", 0))
            ep_inter_ratios.append(mets.get("inter_edge_ratio", 0))
            ep_densities.append(mets.get("graph_density", 0))   # topology_stability
            pbar.set_postfix(loss=f"{loss.item():.3f}", acc=f"{acc:.3f}",
                             density=f"{mets['graph_density']:.2f}",
                             intra=f"{mets.get('intra_edge_ratio', 0):.2f}",
                             nan_skip=nan_skips)

        scheduler.step()

        # ── Validation ────────────────────────────────────────────────────────
        val_acc_mean = 0.0
        if epoch % cfg["eval_every"] == 0:
            model.eval()
            val_accs = []
            with torch.no_grad():
                for _ in range(cfg["n_eval_episodes"]):
                    _, acc, _ = run_episode(
                        model, episode_fn_val,
                        cfg["n_way"], cfg["k_shot"], cfg["n_query"])
                    val_accs.append(acc)
            val_acc_mean = float(np.mean(val_accs))
            val_ci       = 1.96 * float(np.std(val_accs)) / (len(val_accs) ** 0.5)

            # ── Checkpoint best model → Drive ─────────────────────────────────
            if val_acc_mean > best_val_acc:
                best_val_acc = val_acc_mean
                ckpt_path    = CKPT_DIR / f"{run_name}_best.pth"
                torch.save({
                    "epoch"   : epoch,
                    "state"   : model.state_dict(),
                    "val_acc" : best_val_acc,
                    "cfg"     : cfg,
                }, str(ckpt_path))
                print(f"  ✓ New best val acc = {best_val_acc:.4f}  (ckpt saved)")

        epoch_time = time.time() - t_start
        topology_stability = float(np.std(ep_densities)) if ep_densities else 0.0
        log = {
            "epoch"             : epoch,
            "train_loss"        : float(np.mean(train_losses)),
            "train_acc"         : float(np.mean(train_accs)),
            "val_acc"           : val_acc_mean,
            "grad_norm_mean"    : float(np.mean(grad_norms)),
            "lambda_edge_scale" : lambda_edge_scale,
            "use_dynamic"       : model.use_dynamic,
            "graph_density"     : mets.get("graph_density", 0),
            "avg_degree"        : mets.get("avg_degree", 0),
            "degree_std"        : mets.get("degree_std", 0),      # plan §6.1 & §6.2
            "topology_stability": topology_stability,              # plan §6.1: density std across episodes
            "edge_entropy"      : mets.get("edge_entropy", 0),
            "layer_stability"   : mets.get("layer_stability", []),
            "intra_edge_ratio"  : float(np.mean(ep_intra_ratios)),
            "inter_edge_ratio"  : float(np.mean(ep_inter_ratios)),
            "num_parameters"    : sum(p.numel() for p in model.parameters()),  # plan §6.2
            "nan_skipped"       : nan_skips,
            "epoch_time_s"      : epoch_time,
        }
        # Density-collapse early warning
        if log["graph_density"] < 0.01 and model.use_dynamic:
            print(f"  ⚠ Epoch {epoch}: graph density collapsed to "
                  f"{log['graph_density']:.4f} — check lambda_sparse / adj_residual")
        history.append(log)

        # JSON backup to Drive every epoch
        with open(str(RESULTS_DIR / f"{run_name}_history.json"), "w") as f:
            json.dump(history, f)

        if log_wandb:
            import wandb
            wandb.log(log)

        if epoch % cfg["eval_every"] == 0:
            print(f"Epoch {epoch:4d} | train_acc={log['train_acc']:.4f} "
                  f"val_acc={val_acc_mean:.4f}±{val_ci:.4f} "
                  f"density={log['graph_density']:.3f} "
                  f"intra={log['intra_edge_ratio']:.3f} "
                  f"nan_skip={nan_skips} "
                  f"t={epoch_time:.1f}s")
            _health_check(log, history, epoch, cfg)

    print(f"\nTraining complete. Best val acc: {best_val_acc:.4f}")
    return model, history


print("Training loop defined. Call train(CFG, ...) to start.")
print("Model parameter count:", sum(p.numel() for p in build_model(CFG).parameters()))






    print(f"\nTraining complete. Best val acc: {best_val_acc:.4f}")print("Model parameter count:", sum(p.numel() for p in build_model(CFG).parameters()))

print("Training loop defined. Call train(CFG, ...) to start.")
    return model, historyprint("Training loop defined. Call train(CFG, ...) to start.")

print("Model parameter count:", sum(p.numel() for p in build_model(CFG).parameters()))





IndentationError: unexpected indent (3868375125.py, line 256)

## Section 12b: Execute Training (CIFAR-FS, 5-way 1-shot)

In [15]:
# ── Section 12b: Execute Training ────────────────────────────────────────────
# Run this cell to start real training on CIFAR-FS.
# Checkpoints are saved to Google Drive every time val_acc improves.
# Training history is also JSON-backed to Drive after every epoch.
#
# Estimated time on Colab T4 GPU:
#   200 epochs × 300 episodes ≈ 120–150 min total
#   (extended from 100 — model was still improving at epoch 100 in prior run)
#
# Key changes vs. prior run:
#   - CosineAnnealingLR replaces StepLR (smoother decay, no premature collapse)
#   - lambda_sparse reduced 0.01 → 0.005 (target density ~0.10–0.15)
#   - n_epochs extended 100 → 200
#   - intra_edge_ratio logged per epoch (diagnostic for edge loss effectiveness)
#
# ── Optional: enable W&B cloud logging ───────────────────────────────────────
# Uncomment the two lines below to sync metrics to Weights & Biases.
# init_wandb("dekae_cifarfs_5way1shot")
# LOG_WANDB = True

LOG_WANDB = False   # set True after calling init_wandb()

# ── Training config (edit here to override CFG) ──────────────────────────────
# CFG is already defined in Section 12. Override individual keys here if needed.
# To do a quick iteration: CFG["n_epochs"] = 50

print("=" * 60)
print("DEKAE Training — CIFAR-FS  5-way 1-shot")
print("=" * 60)
print(f"  epochs          : {CFG['n_epochs']}")
print(f"  episodes/epoch  : {CFG['n_episodes_train']}")
print(f"  warm-up epochs  : {CFG['t_warm']}")
print(f"  edge-loss ramp  : {CFG['edge_loss_ramp']} epochs")
print(f"  sparsity mode   : {CFG['sparsity_mode']}  (lambda_sparse={CFG['lambda_sparse']})")
print(f"  lambda_edge     : {CFG['lambda_edge']}")
print(f"  use_edge_proj   : {CFG['use_edge_proj']}")
print(f"  scheduler       : CosineAnnealingLR (T_max={CFG['n_epochs']}, eta_min=1e-6)")
print(f"  device          : {DEVICE}")
print("=" * 60)

trained_model, train_history = train(
    cfg              = CFG,
    episode_fn_train = train_sampler_aug,    # augmented CIFAR-FS train split
    episode_fn_val   = val_sampler,          # CIFAR-FS val split
    run_name         = "dekae_cifarfs",
    log_wandb        = LOG_WANDB,
)

best_val = max((e["val_acc"] for e in train_history if e["val_acc"] > 0), default=0.0)
print(f"\n✓ Training complete — best val accuracy: {best_val * 100:.2f}%")
print(f"  Checkpoints → {CKPT_DIR}/dekae_cifarfs_best.pth")
print(f"  History     → {RESULTS_DIR}/dekae_cifarfs_history.json")


DEKAE Training — CIFAR-FS  5-way 1-shot


NameError: name 'CFG' is not defined

## Section 13: Evaluation & Statistical Testing

In [16]:
# ── Section 13: Evaluation & Statistical Testing ─────────────────────────────
from scipy import stats as scipy_stats

def evaluate(model: DEKAEModel, episode_fn, n_way: int, k_shot: int,
             n_query: int, n_episodes: int = 1000) -> dict:
    """
    Evaluate over n_episodes test episodes.
    Returns accuracy mean, 95% CI, and full per-episode result dict.
    """
    model.eval()
    accs, densities, avg_degrees = [], [], []
    intra_ratios, inter_ratios   = [], []

    with torch.no_grad():
        for _ in tqdm(range(n_episodes), desc="Evaluating"):
            s_imgs, s_lbl, q_imgs, q_lbl = episode_fn(n_way, k_shot, n_query)
            s_imgs = s_imgs.to(DEVICE)
            s_lbl  = s_lbl.to(DEVICE)
            q_imgs = q_imgs.to(DEVICE)
            q_lbl  = q_lbl.to(DEVICE)

            logits, _, mets = model(s_imgs, s_lbl, q_imgs)
            preds = logits.argmax(dim=1)
            acc   = (preds == q_lbl).float().mean().item()
            accs.append(acc)
            densities.append(mets["graph_density"])
            avg_degrees.append(mets["avg_degree"])

    accs_np  = np.array(accs)
    mean_acc = accs_np.mean()
    ci_95    = 1.96 * accs_np.std() / (len(accs_np) ** 0.5)

    return {
        "mean_acc"    : mean_acc,
        "ci_95"       : ci_95,
        "per_episode" : accs,
        "avg_density" : np.mean(densities),
        "avg_degree"  : np.mean(avg_degrees),
    }


def paired_ttest(accs_a: list, accs_b: list) -> dict:
    """
    Paired t-test between two models across the same episodes.
    Returns t-statistic and p-value. p < 0.05 → significant improvement.
    """
    t_stat, p_val = scipy_stats.ttest_rel(accs_b, accs_a)
    return {"t_statistic": t_stat, "p_value": p_val,
            "significant_at_0.05": p_val < 0.05}


def compute_intra_inter_edge_ratio(A: torch.Tensor, labels: torch.Tensor):
    """
    Given an adjacency matrix and node labels, compute the ratio of
    intra-class and inter-class edges (for graph quality reporting).
    """
    N = A.size(0)
    intra = inter = 0
    for i in range(N):
        for j in range(N):
            if i != j and A[i, j] > 0:
                if labels[i] == labels[j]:
                    intra += 1
                else:
                    inter += 1
    total = intra + inter
    return intra / max(total, 1), inter / max(total, 1)


# ── Example usage (illustrative — replace dummy episode_fn with real loader) ──
def _dummy_episode_fn(n_way, k_shot, n_query):
    """Placeholder using synthetic data — replace with real dataset loader."""
    s_imgs  = torch.randn(n_way * k_shot,  3, 84, 84)
    s_lbl   = torch.repeat_interleave(torch.arange(n_way), k_shot)
    q_imgs  = torch.randn(n_way * n_query, 3, 84, 84)
    q_lbl   = torch.repeat_interleave(torch.arange(n_way), n_query)
    return s_imgs, s_lbl, q_imgs, q_lbl

_eval_result = evaluate(_model, _dummy_episode_fn, n_way=5, k_shot=1,
                        n_query=15, n_episodes=50)
print(f"Accuracy: {_eval_result['mean_acc']*100:.2f} ± "
      f"{_eval_result['ci_95']*100:.2f}% (95% CI)")
print(f"Avg density: {_eval_result['avg_density']:.3f}")

NameError: name '_model' is not defined

In [17]:
# ── Section 13b: Real Test-Set Evaluation ────────────────────────────────────
# Run AFTER Section 12b training completes.
# Evaluates the best checkpoint on the CIFAR-FS test split.
# Runs 600 episodes for stable 95% CI (plan requirement: ≥ 600 episodes).

import os

# ── Load best checkpoint if available ────────────────────────────────────────
ckpt_path = CKPT_DIR / "dekae_cifarfs_best.pth"
if os.path.exists(str(ckpt_path)):
    ckpt = torch.load(str(ckpt_path), map_location=DEVICE)
    eval_model = build_model(ckpt["cfg"])
    eval_model.load_state_dict(ckpt["state"])
    eval_model.set_use_dynamic(True)
    print(f"Loaded checkpoint from epoch {ckpt['epoch']}  "
          f"(val_acc={ckpt['val_acc']*100:.2f}%)")
else:
    # No checkpoint yet — use the in-memory trained model from Section 12b
    eval_model = trained_model
    print("No checkpoint found — using in-memory trained_model.")

# ── 5-way 1-shot on test split (600 episodes) ────────────────────────────────
print("\nRunning 5-way 1-shot evaluation on CIFAR-FS test split (600 episodes)…")
test_result_1shot = evaluate(eval_model, test_sampler,
                             n_way=5, k_shot=1, n_query=15,
                             n_episodes=600)
print(f"\n  5-way  1-shot: {test_result_1shot['mean_acc']*100:.2f} "
      f"± {test_result_1shot['ci_95']*100:.2f}%  (95% CI)")
print(f"  Avg graph density: {test_result_1shot['avg_density']:.3f}")
print(f"  Avg node degree  : {test_result_1shot['avg_degree']:.2f}")

# ── 5-way 5-shot on test split (600 episodes) ─────────────────────────────────
# 5-shot tests whether edge correction loss now contributes (≥2 support/class)
test_sampler_5shot = EpisodicSampler(test_dataset, n_way=5, k_shot=5, n_query=15)
print("\nRunning 5-way 5-shot evaluation on CIFAR-FS test split (600 episodes)…")
test_result_5shot = evaluate(eval_model, test_sampler_5shot,
                             n_way=5, k_shot=5, n_query=15,
                             n_episodes=600)
print(f"\n  5-way  5-shot: {test_result_5shot['mean_acc']*100:.2f} "
      f"± {test_result_5shot['ci_95']*100:.2f}%  (95% CI)")

# ── Summary table ─────────────────────────────────────────────────────────────
print("\n" + "=" * 55)
print(f"{'Method':<30} {'5-way 1-shot':>12}  {'5-way 5-shot':>12}")
print("-" * 55)
print(f"{'DEKAE (ours)':<30} "
      f"{test_result_1shot['mean_acc']*100:>10.2f}%  "
      f"{test_result_5shot['mean_acc']*100:>10.2f}%")
print("=" * 55)
print("Note: Compare against FSAKE reported: 68.9% (1-shot) on miniImageNet.")
print("CIFAR-FS is typically ~5-8% higher than miniImageNet for the same method.")

# ── Save results to Drive ─────────────────────────────────────────────────────
import json
final_results = {
    "dataset"     : ACTIVE_DATASET,
    "1shot_acc"   : test_result_1shot["mean_acc"],
    "1shot_ci95"  : test_result_1shot["ci_95"],
    "5shot_acc"   : test_result_5shot["mean_acc"],
    "5shot_ci95"  : test_result_5shot["ci_95"],
    "avg_density" : test_result_1shot["avg_density"],
}
with open(str(RESULTS_DIR / "dekae_test_results.json"), "w") as f:
    json.dump(final_results, f, indent=2)
print(f"\nResults saved → {RESULTS_DIR}/dekae_test_results.json")

NameError: name 'CKPT_DIR' is not defined

## Section 14: Synthetic Graph Recovery Experiment (Section 2.3)

Planted-partition protocol: noise level $\sigma$ calibrated so k-NN achieves ~65% Graph F1, leaving room for DEKAE to demonstrate genuine topology recovery.

In [18]:
# ── Section 14: Synthetic Graph Recovery Experiment ──────────────────────────
from sklearn.metrics import precision_score, recall_score, f1_score

def topology_recovery_metrics(A_pred: torch.Tensor,
                               A_star: torch.Tensor,
                               threshold: float = 0.5) -> dict:
    """
    Compare predicted adjacency against ground-truth planted adjacency.

    threshold : binarisation cutoff.
        - Use 0.5 for DEKAE (sigmoid outputs in (0,1)).
        - Use 1e-9 for k-NN (row-normalised values ~1/k, never > 0.5).
    """
    A_pred_bin = (A_pred > threshold).float()
    # Remove diagonal
    N = A_pred.size(0)
    mask = 1 - torch.eye(N, device=A_pred.device)
    p = A_pred_bin[mask.bool()].cpu().numpy()
    g = A_star[mask.bool()].cpu().numpy()

    precision = precision_score(g, p, zero_division=0)
    recall    = recall_score(g, p, zero_division=0)
    f1        = f1_score(g, p, zero_division=0)
    return {"edge_precision": precision, "edge_recall": recall, "graph_f1": f1}


def knn_recovery_baseline(X: torch.Tensor, A_star: torch.Tensor, k: int):
    """Run static k-NN and measure topology recovery.
    Uses threshold=1e-9 because row-normalised k-NN values are ~1/k << 0.5.
    """
    A_knn = build_knn_adjacency(X, k=k)
    return topology_recovery_metrics(A_knn, A_star, threshold=1e-9)


def synthetic_recovery_experiment(n_way=5, n_per_class=5, feat_dim=64,
                                   sigma_values=(0.3, 0.6, 0.8, 1.0),
                                   n_trials=50, knn_k=5,
                                   trained_model: "DEKAEModel | None" = None):
    """
    Run the topology-recovery experiment across multiple noise levels.

    trained_model : if provided, use the trained model's edge/dynamic modules
                    for DEKAE F1 measurement (post-training fidelity test).
                    When None, uses freshly-initialised modules (pre-training baseline).

    NOTE: The pre-training baseline (trained_model=None) is expected to have
    near-zero F1 — this only establishes the random-init lower bound.
    The meaningful result is the post-training run using the saved checkpoint.

    Returns a pandas DataFrame for easy tabulating.
    """
    try:
        import pandas as pd

        # If trained model is provided, use its embed_dim and first-layer modules
        if trained_model is not None:
            feat_dim = trained_model.embed_dim
            _eim_fixed = trained_model.edge_modules[0]
            _dtm_fixed = trained_model.dynamic_modules[0]
            trained_model.eval()
            print(f"Using trained model modules (feat_dim={feat_dim})")
        else:
            _eim_fixed = None
            _dtm_fixed = None
            print(f"Using fresh (untrained) modules (feat_dim={feat_dim})")

        results = []
        for sigma in sigma_values:
            knn_f1s, ours_f1s = [], []
            for _ in range(n_trials):
                X, y, A_star = synthetic_episode(
                    n_way=n_way, n_nodes_per_class=n_per_class,
                    feat_dim=feat_dim, sigma=sigma)

                # k-NN baseline (threshold=1e-9 — row-normalised values are ~0.2)
                knn_met = knn_recovery_baseline(X, A_star, k=knn_k)
                knn_f1s.append(knn_met["graph_f1"])

                # DEKAE topology recovery
                A_init = build_knn_adjacency(X, k=knn_k)
                with torch.no_grad():
                    if _eim_fixed is not None:
                        # Use trained model's learned modules
                        E_, src_, dst_ = _eim_fixed(X, A_init)
                        A_dek, _       = _dtm_fixed(X, E_, src_, dst_,
                                                     sparsity_reg=False)
                    else:
                        # Fresh random modules (pre-training baseline)
                        eim_tmp = EdgeIncidenceModule(feat_dim, feat_dim).to(X.device)
                        dtm_tmp = DynamicTopologyModule(feat_dim, rank=8,
                                                        edge_dim=feat_dim).to(X.device)
                        E_, src_, dst_ = eim_tmp(X, A_init)
                        A_dek, _       = dtm_tmp(X, E_, src_, dst_,
                                                  sparsity_reg=False)
                ours_met = topology_recovery_metrics(A_dek.detach(), A_star)
                ours_f1s.append(ours_met["graph_f1"])

            results.append({
                "sigma"        : sigma,
                "kNN_F1_mean"  : round(np.mean(knn_f1s), 4),
                "kNN_F1_std"   : round(np.std(knn_f1s), 4),
                "DEKAE_F1_mean": round(np.mean(ours_f1s), 4),
                "DEKAE_F1_std" : round(np.std(ours_f1s), 4),
            })

        df = pd.DataFrame(results)
        print(df.to_string(index=False))
        return df
    except ImportError:
        print("pandas not available — run: !pip install pandas")
        return None


# ── Run 1: pre-training baseline (lower bound — expected near-zero F1) ───────
# This establishes the random-init baseline only. Do NOT interpret this as
# the model's topology recovery capability. Run 2 (below) is the real test.
print("=== Synthetic topology-recovery: untrained model (lower bound) ===")
df_recovery_pretrain = synthetic_recovery_experiment(
    n_way=5, n_per_class=5, feat_dim=64,
    sigma_values=[0.3, 0.6, 0.8, 1.0],
    n_trials=30, knn_k=5,
    trained_model=None,
)

# ── Run 2: post-training (MEANINGFUL result — run AFTER Section 12b) ─────────
# Uses the trained checkpoint loaded in Section 13b.
# If the trained model is not loaded yet, this block is skipped automatically.
# To run manually: set FORCE_RUN2 = True after loading eval_model.
FORCE_RUN2 = False   # ← set True after training; or it auto-runs if eval_model exists

if FORCE_RUN2 or "eval_model" in dir():
    _trained = eval_model if "eval_model" in dir() else None
    if _trained is not None:
        print("\n=== Synthetic topology-recovery: TRAINED model (meaningful result) ===")
        df_recovery_trained = synthetic_recovery_experiment(
            n_way=5, n_per_class=5, feat_dim=128,   # feat_dim matches trained backbone
            sigma_values=[0.3, 0.6, 0.8, 1.0],
            n_trials=30, knn_k=5,
            trained_model=_trained,
        )

        # Compare pre vs post training
        if df_recovery_pretrain is not None and df_recovery_trained is not None:
            import pandas as pd
            df_compare = df_recovery_pretrain[["sigma", "kNN_F1_mean", "DEKAE_F1_mean"]].copy()
            df_compare.columns = ["sigma", "kNN_F1", "DEKAE_untrained_F1"]
            df_compare["DEKAE_trained_F1"] = df_recovery_trained["DEKAE_F1_mean"].values
            print("\n=== Pre vs Post Training Topology Recovery ===")
            print(df_compare.to_string(index=False))
            df_compare.to_csv(str(RESULTS_DIR / "topology_recovery_comparison.csv"), index=False)
            print(f"\nSaved → {RESULTS_DIR}/topology_recovery_comparison.csv")
    else:
        print("eval_model not found. Load checkpoint in Section 13b first, then re-run.")
else:
    print("\n  Run 2 (trained model) deferred. Load checkpoint in Section 13b and "
          "set FORCE_RUN2=True to execute.")


=== Synthetic topology-recovery: untrained model (lower bound) ===
Using fresh (untrained) modules (feat_dim=64)


NameError: name 'synthetic_episode' is not defined

## Section 15: Ablation Study Runner

Each group (A–G from the plan) can be run by calling `run_ablation(cfg_dict)`. Configs are defined per variant and results collected into a comparison table.

In [19]:
# ── Section 15: Ablation Study Runner ────────────────────────────────────────
import copy

# ── Base config (derived from CFG) ───────────────────────────────────────────
# IMPORTANT: extended from 30→100 epochs and 100→300 ep/epoch so ablation
# results are comparable to the full training run (30 epochs was too short
# to rank variants reliably — all differences were within CI).
BASE = copy.deepcopy(CFG)
BASE["n_epochs"]          = 100
BASE["n_episodes_train"]  = 300
BASE["n_eval_episodes"]   = 200

# ──────────────────────────────────────────────────────────────────────────────
# Group A: Topology Dynamics  (plan §5.1)
#
# Ablation table (matches plan §5.1):
#   A0 — ProtoNet anchor (no GNN)
#   A1 — FSAKE: static k-NN, node aggregation A'H (Case 1, use_case2=False)
#   A2 — dynamic rewiring only (no edge-to-node proj)
#   A3 — dynamic + Case 1 node update (X' = σ(g(E))·A'H)   [plan A4]
#   A4 — dynamic + Case 2 node update (X' = A_edge@E@W')   [plan A5]
#   A5 — dynamic + Case 2 + edge loss (full, no LMT)        [plan A6]
#
# NOTE: All Group A variants use use_lmt=False so the GNN contribution is
# isolated.  Group H tests the LMT module independently (H1a vs H1b).
#
# NOTE on A2 vs A3 distinction (bug fixed from prior run):
#   A2 = dynamic rewiring only → use_edge_proj=False (simple A'H message passing)
#   A3 = dynamic + Case 1 edge-gated node agg → use_edge_proj=True, use_case2=False
#   Previously A2 and A3 had identical configs. Now correctly separated.
# ──────────────────────────────────────────────────────────────────────────────
ABLATION_CONFIGS = {

    # A0 — ProtoNet anchor: no GNN at all (pure prototype matching on backbone)
    # use_dynamic=False, use_edge_proj=False → A_init @ H only (1 MP pass)
    "A0_ProtoNet": {**BASE,
        "use_dynamic": False, "use_edge_proj": False,
        "use_case2": False, "use_lmt": False,
        "lambda_edge": 0.0, "sparsity_mode": "none"},

    # A1 — FSAKE baseline (static k-NN, Case 1 node aggregation, no edge loss)
    # use_case2=False: node update is edge-gated aggregation of neighbour features
    # FSAKE uses node-level correction — here reproduced as static graph + edge KF
    "A1_FSAKE": {**BASE,
        "use_dynamic": False, "use_edge_proj": True,
        "use_case2": False,   "use_lmt": False,
        "lambda_edge": 0.0, "sparsity_mode": "none"},

    # A2 — Dynamic rewiring only (no edge-to-node projection, no edge loss)
    "A2_dyn_only": {**BASE,
        "use_dynamic": True, "use_edge_proj": False,
        "use_case2": True,   "use_lmt": False,   # use_case2 irrelevant when use_edge_proj=False
        "lambda_edge": 0.0},

    # A3 — Dynamic + Case 1: X' = σ(g(E)) · A' @ H  (edge-gated node aggregation)
    # [plan §5.1 A4]  isolates the benefit of edge-gated vs. plain message passing
    "A3_case1": {**BASE,
        "use_dynamic": True, "use_edge_proj": True,
        "use_case2": False,  "use_lmt": False,
        "lambda_edge": 0.0},

    # A4 — Dynamic + Case 2: X' = A_edge @ E @ W'  (edge-vector node update)
    # [plan §5.1 A5]  the stronger novelty — nodes updated from edge vectors
    "A4_case2": {**BASE,
        "use_dynamic": True, "use_edge_proj": True,
        "use_case2": True,   "use_lmt": False,
        "lambda_edge": 0.0},

    # A5 — Full DEKAE model (dynamic + Case 2 + supervised edge loss, no LMT)
    # [plan §5.1 A6]  direct comparison: shows edge loss adds value beyond Case 2
    "A5_DEKAE_noLMT": {**BASE,
        "use_dynamic": True, "use_edge_proj": True,
        "use_case2": True,   "use_lmt": False,
        "lambda_edge": 0.5},

    # A6 — Full DEKAE with LMT (complete proposed model)
    # [Group H anchor] — break-even comparison against A5 to isolate LMT gain
    "A6_DEKAE_full": {**BASE,
        "use_dynamic": True, "use_edge_proj": True,
        "use_case2": True,   "use_lmt": True,
        "lambda_edge": 0.5},

    # ── Group B: Variable Degree ──────────────────────────────────────────────
    "B1_fixed_k5":  {**BASE, "use_dynamic": False, "use_lmt": False,
                     "topk_k": 5,  "lambda_edge": 0.0},
    "B2_fixed_k10": {**BASE, "use_dynamic": False, "use_lmt": False,
                     "topk_k": 10, "lambda_edge": 0.0},
    "B3_dynamic":   {**BASE, "use_dynamic": True,  "use_lmt": False,
                     "lambda_edge": 0.5},

    # ── Group E: Sparsity ─────────────────────────────────────────────────────
    # ⚠ WARNING: E4_laplacian is retained for completeness but empirically
    # collapsed to ~26% accuracy (near random chance) in initial runs.
    # Laplacian smoothness harmonises adjacent node representations — in a
    # mixed-class few-shot episode this destroys class discriminability.
    # Do NOT use as the primary model — include only as a negative ablation.
    "E1_no_sparse":   {**BASE, "sparsity_mode": "none",    "use_lmt": False},
    "E2_l1":          {**BASE, "sparsity_mode": "l1",      "use_lmt": False},
    "E3_topk":        {**BASE, "sparsity_mode": "topk",    "use_lmt": False},
    "E4_laplacian":   {**BASE, "sparsity_mode": "laplacian","use_lmt": False},  # ⚠ known-harmful

    # ── Group F: Edge MLP Capacity ────────────────────────────────────────────
    "F1_full_rank64": {**BASE, "rank": 64,  "use_lmt": False},
    "F2_lowrank16":   {**BASE, "rank": 16,  "use_lmt": False},
    "F3_dotprod1":    {**BASE, "rank": 1,   "use_lmt": False},

    # ── Group G5: Lambda_edge Sensitivity (plan §5.7 G5) ─────────────────────
    # NOTE: Implementation names kept as G4 for continuity, but these correspond
    # to plan §5.7 G5 (Edge Loss Weight Sensitivity).
    "G5_lam0":   {**BASE, "lambda_edge": 0.0, "use_lmt": False},
    "G5_lam01":  {**BASE, "lambda_edge": 0.1, "use_lmt": False},
    "G5_lam05":  {**BASE, "lambda_edge": 0.5, "use_lmt": False},
    "G5_lam1":   {**BASE, "lambda_edge": 1.0, "use_lmt": False},
    "G5_lam2":   {**BASE, "lambda_edge": 2.0, "use_lmt": False},

    # ── Group H: Latent Mediator Transformer Ablation (plan §5.8) ────────────
    #
    # H1: With vs. Without LMT (use A5_DEKAE_noLMT vs A6_DEKAE_full above)
    #
    # H2: Number of mediator tokens m (plan §5.8 H2)
    # Expected: moderate m=8-16 best; m=N_s collapses to near-direct attention
    "H2_m2":     {**BASE, "use_lmt": True, "n_mediators": 2,   "lambda_edge": 0.5},
    "H2_m8":     {**BASE, "use_lmt": True, "n_mediators": 8,   "lambda_edge": 0.5},  # default
    "H2_m16":    {**BASE, "use_lmt": True, "n_mediators": 16,  "lambda_edge": 0.5},
    "H2_m32":    {**BASE, "use_lmt": True, "n_mediators": 32,  "lambda_edge": 0.5},  # ≈N_s (5-shot)

    # H3: LMT depth — number of gather-distribute cycles (plan §5.8 H3)
    # Expected: 3 layers > 1 layer (iterative refinement adds value)
    "H3_L1":     {**BASE, "use_lmt": True, "lmt_layers": 1, "lambda_edge": 0.5},
    "H3_L3":     {**BASE, "use_lmt": True, "lmt_layers": 3, "lambda_edge": 0.5},  # default
    "H3_L5":     {**BASE, "use_lmt": True, "lmt_layers": 5, "lambda_edge": 0.5},

    # ── Group H4: Mediator Initialisation Strategy (plan §5.8 H4) ─────────────
    # Tests whether learned mediator tokens contribute vs. fixed/zero baselines.
    # Expected: 'learned' > 'fixed' > 'zero' (meta-prior in M is beneficial)
    "H4_learned": {**BASE, "use_lmt": True, "lmt_init_strategy": "learned",
                   "lambda_edge": 0.5},                              # default
    "H4_fixed":   {**BASE, "use_lmt": True, "lmt_init_strategy": "fixed",
                   "lambda_edge": 0.5},                              # random, no grad
    "H4_zero":    {**BASE, "use_lmt": True, "lmt_init_strategy": "zero",
                   "lambda_edge": 0.5},                              # zeros init, no grad

    # ── Group H5: LMT Phase Ablation (plan §5.8 H5) ───────────────────────────
    # Tests whether the gather phase, distribute phase, or both are necessary.
    # Expected: 'both' >> 'gather_only' ≈ 'distribute_only'
    "H5a_gather_only":     {**BASE, "use_lmt": True, "lmt_phases": "gather_only",
                             "lambda_edge": 0.5},  # mediator updated, not broadcast back
    "H5b_distribute_only": {**BASE, "use_lmt": True, "lmt_phases": "distribute_only",
                             "lambda_edge": 0.5},  # broadcast from init M, no gather
    "H5c_both":            {**BASE, "use_lmt": True, "lmt_phases": "both",
                             "lambda_edge": 0.5},  # full gather + distribute (default)

    # ── Group I: Multi-Level Edge Supervision (plan §5.9) ─────────────────────
    # Tests where in the GNN stack to apply the edge correction loss.
    # I1 — penultimate only: edge loss applied only at the final GNN layer
    # I2 — all layers equal: edge loss at every layer with equal weight (current default)
    # I3 — all layers decaying: edge loss at all layers, weight = 1/(L-l)
    #       (higher weight at later layers, tapering toward earlier ones)
    # Expected: I2 ≈ I3 > I1 (multi‑layer supervision beneficial)
    "I1_penultimate":  {**BASE, "use_lmt": True, "edge_loss_mode": "penultimate",
                        "lambda_edge": 0.5},
    "I2_all_equal":    {**BASE, "use_lmt": True, "edge_loss_mode": "all",
                        "lambda_edge": 0.5},     # default behaviour
    "I3_all_decaying": {**BASE, "use_lmt": True, "edge_loss_mode": "decaying",
                        "lambda_edge": 0.5},
}


def run_ablation(name: str, cfg: dict, episode_fn_train, episode_fn_val,
                 log_wandb: bool = False):
    """
    Train and evaluate a single ablation variant.
    Applies ALL ablation flags from cfg (use_dynamic, use_edge_proj,
    use_case2, use_lmt) so ABLATION_CONFIGS fully controls the model.
    Returns a summary dict with name, mean_acc, ci_95.
    """
    print(f"\n{'='*60}")
    print(f"Running ablation: {name}")
    print(f"{'='*60}")
    set_seed(cfg.get("seed", 42))

    model = build_model(cfg)
    # Apply all ablation toggle flags from config after build
    # (build_model already passes them; these calls are an explicit safety guard)
    model.set_use_dynamic(cfg.get("use_dynamic", True))
    model.set_use_edge_proj(cfg.get("use_edge_proj", True))
    model.set_use_case2(cfg.get("use_case2", True))     # plan §5.1: Case 1 vs Case 2
    model.set_use_lmt(cfg.get("use_lmt", True))         # plan §5.8: Group H ablation

    trained_model, history = train(
        cfg, episode_fn_train, episode_fn_val,
        run_name=name, log_wandb=log_wandb
    )

    # Evaluate
    res = evaluate(trained_model, episode_fn_val,
                   cfg["n_way"], cfg["k_shot"], cfg["n_query"],
                   n_episodes=cfg.get("n_eval_episodes", 200))
    res["name"] = name
    print(f"→ {name}: {res['mean_acc']*100:.2f} ± {res['ci_95']*100:.2f}%")
    return res


def run_all_ablations(groups: list, episode_fn_train, episode_fn_val):
    """Run a subset of ablation groups and collect results."""
    results = []
    for name in groups:
        if name not in ABLATION_CONFIGS:
            print(f"⚠ Unknown ablation key: {name}")
            continue
        res = run_ablation(name, ABLATION_CONFIGS[name],
                           episode_fn_train, episode_fn_val)
        results.append(res)
    return results


print("Ablation configs defined.")
print("Group A variants :", [k for k in ABLATION_CONFIGS if k.startswith("A")])
print("Group E variants :", [k for k in ABLATION_CONFIGS if k.startswith("E")])
print("Group H variants :", [k for k in ABLATION_CONFIGS if k.startswith("H")])
print("Group H4 variants:", [k for k in ABLATION_CONFIGS if k.startswith("H4")])
print("Group H5 variants:", [k for k in ABLATION_CONFIGS if k.startswith("H5")])
print("Group G5 variants:", [k for k in ABLATION_CONFIGS if k.startswith("G5")])
print("Group I  variants:", [k for k in ABLATION_CONFIGS if k.startswith("I")])
print(f"\nBase ablation config: {BASE['n_epochs']} epochs × {BASE['n_episodes_train']} ep/epoch "
      f"= {BASE['n_epochs'] * BASE['n_episodes_train']:,} episodes total")
print("\nAll Group A variants have use_lmt=False (LMT tested separately in Group H).")
print("A3_case1 (Case 1) and A4_case2 (Case 2) now correctly differ by use_case2 flag.")


NameError: name 'CFG' is not defined

In [20]:
# ── Section 15b: Execute Ablation Groups A, E, and H ─────────────────────────
# Run AFTER Section 12b training completes.
# Each variant trains for 100 epochs × 300 episodes ≈ 30,000 episodes total.
#
# Estimated time per variant on T4: ~60–90 min
# Total for Groups A+E core (8 variants, E4 excluded): ~8–10 hours on T4.
# Recommended: run one group at a time, saving results between sessions.
#
# Group A proves: dynamic > static, Case 2 > Case 1, edge loss adds value,
#                 and all Group A variants use use_lmt=False (fair GNN-only comparison).
# Group E proves: sparsity prevents collapse; E4_laplacian is excluded from the
#                 default run — it collapsed to ~26% in initial experiments.
# Group H proves: LMT adds value on top of the best GNN-only model (A5_DEKAE_noLMT).

import pandas as pd

def print_ablation_table(results: list, title: str = "Ablation Results"):
    print(f"\n{'='*65}")
    print(f"  {title}")
    print(f"{'='*65}")
    print(f"{'Variant':<22} {'Acc (%)':>9}  {'CI95':>6}  {'Density':>8}  {'Intra-E':>8}")
    print("-" * 65)
    for r in sorted(results, key=lambda x: -x["mean_acc"]):
        print(f"  {r['name']:<20} {r['mean_acc']*100:>8.2f}% "
              f"±{r['ci_95']*100:>5.2f}%  "
              f"{r.get('avg_density', 0):>7.3f}  "
              f"{r.get('avg_intra_ratio', 0):>8.3f}")
    print("=" * 65)


# ── Group A: Topology Dynamics ────────────────────────────────────────────────
# Maps to plan §5.1 ablation table (A0–A6).
# All variants have use_lmt=False so GNN contribution is cleanly isolated.
print("Running Group A ablations (topology dynamics)…")
group_a_results = run_all_ablations(
    groups            = ["A0_ProtoNet", "A1_FSAKE", "A2_dyn_only",
                         "A3_case1", "A4_case2", "A5_DEKAE_noLMT"],
    episode_fn_train  = train_sampler_aug,
    episode_fn_val    = val_sampler,
)
print_ablation_table(group_a_results, "Group A — Topology Dynamics (no LMT)")

# ── Group E: Sparsity Regularisation (E4_laplacian run separately) ───────────
# ⚠ E4_laplacian collapsed to 26.24% in prior run (near random chance).
# Run it last so it doesn't waste compute time before seeing the main results.
print("\nRunning Group E core ablations (sparsity, E4 excluded)…")
group_e_core_results = run_all_ablations(
    groups            = ["E1_no_sparse", "E2_l1", "E3_topk"],
    episode_fn_train  = train_sampler_aug,
    episode_fn_val    = val_sampler,
)
print_ablation_table(group_e_core_results, "Group E — Sparsity Regularisation (core)")

# ── E4_laplacian (run separately, known to be harmful) ───────────────────────
RUN_E4 = False   # ← set True only if you want the negative-ablation data point
if RUN_E4:
    print("\n⚠ Running E4_laplacian — expected to collapse to ~26% (negative ablation)…")
    group_e4_result = run_all_ablations(
        groups            = ["E4_laplacian"],
        episode_fn_train  = train_sampler_aug,
        episode_fn_val    = val_sampler,
    )
    group_e_core_results += group_e4_result
    print_ablation_table(group_e_core_results, "Group E — Sparsity Regularisation (all)")
else:
    print("\n  (E4_laplacian skipped — set RUN_E4=True to include as negative ablation)")

# ── Group H: LMT Ablation (plan §5.8) ────────────────────────────────────────
# H1a = A5_DEKAE_noLMT (baseline, already in Group A above)
# H1b = A6_DEKAE_full  (with LMT)
# H2 = mediator token count sensitivity
# H3 = LMT depth sensitivity
RUN_GROUP_H = False   # ← set True to run LMT ablations (each ~60–90 min)
if RUN_GROUP_H:
    print("\nRunning Group H ablations (LMT module)…")
    # H1: with vs without LMT
    group_h1_results = run_all_ablations(
        groups            = ["A5_DEKAE_noLMT", "A6_DEKAE_full"],
        episode_fn_train  = train_sampler_aug,
        episode_fn_val    = val_sampler,
    )
    print_ablation_table(group_h1_results, "Group H1 — LMT vs No-LMT")

    # H2: mediator count sensitivity
    group_h2_results = run_all_ablations(
        groups            = ["H2_m2", "H2_m8", "H2_m16", "H2_m32"],
        episode_fn_train  = train_sampler_aug,
        episode_fn_val    = val_sampler,
    )
    print_ablation_table(group_h2_results, "Group H2 — Mediator Token Count (m)")

    # H3: LMT depth sensitivity
    group_h3_results = run_all_ablations(
        groups            = ["H3_L1", "H3_L3", "H3_L5"],
        episode_fn_train  = train_sampler_aug,
        episode_fn_val    = val_sampler,
    )
    print_ablation_table(group_h3_results, "Group H3 — LMT Depth (L_lmt)")
else:
    print("\n  (Group H skipped — set RUN_GROUP_H=True after Group A training completes)")

# ── Group H4: Mediator Initialisation Ablation (plan §5.8 H4) ────────────────
RUN_GROUP_H4 = False   # ← set True to test mediator init strategies
if RUN_GROUP_H4:
    print("\nRunning Group H4 ablations (mediator token init strategy)…")
    group_h4_results = run_all_ablations(
        groups            = ["H4_learned", "H4_fixed", "H4_zero"],
        episode_fn_train  = train_sampler_aug,
        episode_fn_val    = val_sampler,
    )
    print_ablation_table(group_h4_results, "Group H4 — Mediator Init Strategy")
else:
    print("  (Group H4 skipped — set RUN_GROUP_H4=True)")

# ── Group H5: LMT Phase Ablation (plan §5.8 H5) ──────────────────────────────
RUN_GROUP_H5 = False   # ← set True to test gather / distribute phases
if RUN_GROUP_H5:
    print("\nRunning Group H5 ablations (LMT phase activation)…")
    group_h5_results = run_all_ablations(
        groups            = ["H5a_gather_only", "H5b_distribute_only", "H5c_both"],
        episode_fn_train  = train_sampler_aug,
        episode_fn_val    = val_sampler,
    )
    print_ablation_table(group_h5_results, "Group H5 — LMT Phase Ablation")
else:
    print("  (Group H5 skipped — set RUN_GROUP_H5=True)")

# ── Group I: Multi-Level Edge Supervision (plan §5.9) ────────────────────────
RUN_GROUP_I = False   # ← set True to test edge-loss application layer
if RUN_GROUP_I:
    print("\nRunning Group I ablations (multi-level edge supervision)…")
    group_i_results = run_all_ablations(
        groups            = ["I1_penultimate", "I2_all_equal", "I3_all_decaying"],
        episode_fn_train  = train_sampler_aug,
        episode_fn_val    = val_sampler,
    )
    print_ablation_table(group_i_results, "Group I — Multi-Level Edge Supervision")
else:
    print("  (Group I skipped — set RUN_GROUP_I=True)")

# ── Save all ablation results to Drive ────────────────────────────────────────
all_ablation = group_a_results + group_e_core_results
with open(str(RESULTS_DIR / "ablation_results.json"), "w") as f:
    import json
    save = [{k: v for k, v in r.items() if k != "per_episode"} for r in all_ablation]
    json.dump(save, f, indent=2)
print(f"\nAblation results saved → {RESULTS_DIR}/ablation_results.json")


Running Group A ablations (topology dynamics)…


NameError: name 'run_all_ablations' is not defined

## Section 16: Metrics Logging (Weights & Biases)

W&B is the recommended logger to survive Colab session disconnects. A JSON backup is saved to Drive at the end of every epoch as a fallback.

In [21]:
# ── Section 16: Metrics Logging (W&B) ────────────────────────────────────────
# Run this cell to initialise W&B before calling train().
# Set LOG_WANDB=True in train() call to enable cloud sync.

try:
    import wandb

    WANDB_PROJECT = "DEKAE-FSL"
    WANDB_ENTITY  = None          # ← set to your W&B username/team if needed

    def init_wandb(run_name: str = "dekae_run", cfg: dict = None):
        wandb.init(
            project = WANDB_PROJECT,
            entity  = WANDB_ENTITY,
            name    = run_name,
            config  = cfg or CFG,
            resume  = "allow",    # resume if run_id matches previous session
        )
        print(f"W&B run initialised: {wandb.run.url}")

    # ── Full metrics schema (plan §6.2) ───────────────────────────────────────
    # Matches the log dict in train() exactly — edit both together if adding keys.
    METRIC_SCHEMA = {
        "train_loss"          : None,   "train_acc"           : None,
        "val_acc"             : None,   "test_acc"            : None,
        "avg_degree"          : None,   "degree_std"          : None,   # plan §6.1 & §6.2
        "edge_entropy"        : None,   "graph_density"       : None,   # watch for collapse
        "topology_stability"  : None,   "intra_edge_ratio"    : None,   # plan §6.1 (density std)
        "inter_edge_ratio"    : None,   "layer_stability"     : None,   # layer_repr_stability
        "grad_norm_mean"      : None,   "lambda_edge_scale"   : None,
        "num_parameters"      : None,   "epoch_time_s"        : None,   # plan §6.2
    }

    print("W&B available. Call init_wandb('run_name') before train().")
    print("Metric keys:", list(METRIC_SCHEMA.keys()))

except ImportError:
    print("W&B not installed. Fallback: JSON logs saved to:", RESULTS_DIR)

# ── Session-disconnect guard: flush & save ────────────────────────────────────
def safe_flush(history: list, run_name: str):
    """Save history to JSON and flush W&B (if active). Call from training loop."""
    path = RESULTS_DIR / f"{run_name}_history.json"
    with open(str(path), "w") as f:
        json.dump(history, f, default=str)
    try:
        import wandb
        if wandb.run is not None:
            wandb.save(str(path))
    except Exception:
        pass


print("safe_flush() registered.")


W&B available. Call init_wandb('run_name') before train().
Metric keys: ['train_loss', 'train_acc', 'val_acc', 'test_acc', 'avg_degree', 'degree_std', 'edge_entropy', 'graph_density', 'topology_stability', 'intra_edge_ratio', 'inter_edge_ratio', 'layer_stability', 'grad_norm_mean', 'lambda_edge_scale', 'num_parameters', 'epoch_time_s']
safe_flush() registered.


## Section 17: Visualization Suite

Five required plots: (1) topology per GNN layer, (2) degree distribution, (3) graph density curve, (4) t-SNE before/after, (5) degree vs. sample difficulty using margin difficulty $d_i = s_1(i) - s_2(i)$.

In [22]:
# ── Section 17: Visualization Suite ──────────────────────────────────────────
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import networkx as nx
from sklearn.manifold import TSNE

sns.set_theme(style="whitegrid", font_scale=1.1)
SAVE_DIR = RESULTS_DIR / "figures"
SAVE_DIR.mkdir(exist_ok=True)


# ── Plot 1: Graph Topology per GNN Layer ─────────────────────────────────────

def plot_topology(A: torch.Tensor, labels: torch.Tensor, title: str = "",
                  save_path=None):
    """Visualise an episode graph coloured by class label."""
    N = A.size(0)
    A_np = (A > 0.01).float().cpu().numpy()
    G    = nx.from_numpy_array(A_np)

    label_list = labels.cpu().tolist()
    cmap = plt.colormaps.get_cmap("tab10")
    colors = [cmap(l / max(max(label_list), 1)) for l in label_list]

    fig, ax = plt.subplots(figsize=(6, 5))
    pos = nx.spring_layout(G, seed=42)
    nx.draw_networkx(G, pos, node_color=colors,
                     with_labels=False, node_size=120,
                     edge_color="#aaa", width=0.6, ax=ax)
    ax.set_title(title)
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=120)
    plt.show()


# ── Plot 2: Degree Distribution Histogram ─────────────────────────────────────

def plot_degree_distribution(A: torch.Tensor, title: str = "", save_path=None):
    A_no_diag = A.clone(); A_no_diag.fill_diagonal_(0)
    degrees = (A_no_diag > 0).float().sum(dim=1).cpu().numpy()
    fig, ax = plt.subplots(figsize=(5, 3))
    ax.hist(degrees, bins=range(int(degrees.max()) + 2),
            color="steelblue", edgecolor="white")
    ax.set_xlabel("Node Degree")
    ax.set_ylabel("Count")
    ax.set_title(title or "Degree Distribution")
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=120)
    plt.show()


# ── Plot 3: Graph Density Curve over Training ─────────────────────────────────

def plot_density_curve(history: list, save_path=None):
    epochs    = [h["epoch"] for h in history]
    densities = [h.get("graph_density", 0) for h in history]
    fig, ax   = plt.subplots(figsize=(6, 3))
    ax.plot(epochs, densities, color="coral", linewidth=2)
    ax.axhline(0.5, ls="--", color="red", alpha=0.6, label="Collapse threshold (0.5)")
    ax.axhspan(0.10, 0.15, alpha=0.08, color="green", label="Target density range (0.10–0.15)")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Graph Density")
    ax.set_title("Graph Density over Training (monitor for collapse)")
    ax.legend()
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=120)
    plt.show()


# ── Plot 3b: Intra/Inter Edge Ratio over Training (NEW) ──────────────────────

def plot_edge_ratio_curve(history: list, save_path=None):
    """
    Plot intra-class vs inter-class edge ratios over training.
    Expected: intra_edge_ratio should INCREASE over training as the model
    learns to preferentially connect same-class nodes via the edge correction
    loss. Flat or decreasing intra ratio = edge loss not engaging.
    """
    epochs = [h["epoch"] for h in history if "intra_edge_ratio" in h]
    if not epochs:
        print("No intra_edge_ratio data in history — re-run training with updated code.")
        return
    intra  = [h["intra_edge_ratio"] for h in history if "intra_edge_ratio" in h]
    inter  = [h["inter_edge_ratio"] for h in history if "inter_edge_ratio" in h]

    fig, ax = plt.subplots(figsize=(7, 3.5))
    ax.plot(epochs, intra, color="#2ecc71", linewidth=2, label="Intra-class edge ratio")
    ax.plot(epochs, inter, color="#e74c3c", linewidth=2, label="Inter-class edge ratio",
            linestyle="--")
    ax.axhline(1.0 / 5, ls=":", color="gray", alpha=0.7,
               label="Random baseline (1/n_way = 0.20)")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Edge Ratio (support-support edges)")
    ax.set_title("Intra/Inter Edge Ratio over Training\n"
                 "(Rising intra ratio = edge loss building class-consistent topology)")
    ax.legend(fontsize=9)
    ax.set_ylim(0, 1)
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=120)
    plt.show()


# ── Plot 4: t-SNE Before / After Dynamic Topology ────────────────────────────

def plot_tsne(H_before: torch.Tensor, H_after: torch.Tensor,
              labels: torch.Tensor, save_path=None):
    H_all  = torch.cat([H_before, H_after], dim=0).cpu().detach().numpy()
    labels_np = labels.cpu().numpy()
    tsne   = TSNE(n_components=2, perplexity=15, random_state=42)
    emb    = tsne.fit_transform(H_all)
    N      = H_before.size(0)

    fig, axes = plt.subplots(1, 2, figsize=(10, 4))
    for ax, title, start, end in zip(
            axes,
            ["Before (H⁰)", "After Dynamic Topology (Hᴸ)"],
            [0, N], [N, 2 * N]):
        sc = ax.scatter(emb[start:end, 0], emb[start:end, 1],
                        c=labels_np, cmap="tab10", s=40, alpha=0.8)
        ax.set_title(title); ax.axis("off")
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=120)
    plt.show()


# ── Plot 5: Node Degree vs. Sample Difficulty ─────────────────────────────────

def plot_degree_vs_difficulty(model: DEKAEModel, episode_fn,
                               n_way: int, k_shot: int, n_query: int,
                               n_episodes: int = 600, save_path=None):
    """
    Bin query nodes into easy / medium / hard tertiles by margin difficulty
    d_i = s1(i) - s2(i)  (smaller margin = harder sample).
    Show median node degree per bin with 95% CI error bars.
    """
    model.eval()
    margin_diffs, degrees = [], []

    with torch.no_grad():
        for _ in tqdm(range(n_episodes), desc="Difficulty analysis"):
            s_imgs, s_lbl, q_imgs, q_lbl = episode_fn(n_way, k_shot, n_query)
            s_imgs = s_imgs.to(DEVICE); s_lbl = s_lbl.to(DEVICE)
            q_imgs = q_imgs.to(DEVICE)

            logits, _, mets = model(s_imgs, s_lbl, q_imgs)

            sorted_logits, _ = logits.sort(dim=1, descending=True)
            margin = (sorted_logits[:, 0] - sorted_logits[:, 1]).cpu().numpy()
            margin_diffs.extend(margin.tolist())

            avg_d = mets.get("avg_degree", 0)
            degrees.extend([avg_d] * logits.size(0))

    margin_arr  = np.array(margin_diffs)
    degree_arr  = np.array(degrees)

    tertile1, tertile2 = np.percentile(margin_arr, [33, 66])
    bins = {"Hard\n(low margin)"  : degree_arr[margin_arr <= tertile1],
            "Medium"              : degree_arr[(margin_arr > tertile1) & (margin_arr <= tertile2)],
            "Easy\n(high margin)" : degree_arr[margin_arr > tertile2]}

    labels_plot  = list(bins.keys())
    means  = [np.median(v) for v in bins.values()]
    cis    = [1.96 * np.std(v) / max(np.sqrt(len(v)), 1) for v in bins.values()]

    fig, ax = plt.subplots(figsize=(5, 4))
    ax.bar(labels_plot, means, yerr=cis, color=["#e74c3c", "#f39c12", "#2ecc71"],
           capsize=6, edgecolor="white")
    ax.set_ylabel("Average Node Degree")
    ax.set_title("Node Degree vs. Sample Difficulty\n(Hard samples → more neighbours)")
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=120)
    plt.show()


# ── Plot 6: Adjacency Heatmap — Static k-NN vs. Learned Dynamic ──────────────

def plot_adjacency_heatmap(A_dynamic: torch.Tensor, A_static: torch.Tensor,
                            labels: torch.Tensor, save_path=None):
    """
    Side-by-side heatmap comparing the static k-NN adjacency to the
    learned dynamic adjacency for the same episode.

    Nodes are sorted by class label so intra-class block structure becomes
    visually apparent.  A well-trained DEKAE model should show stronger
    block-diagonal structure in A_dynamic vs. A_static.

    Plan reference: §6.5 visualisation item 6.

    Parameters
    ----------
    A_dynamic : (N, N) float tensor — learned dynamic adjacency A'(L)
    A_static  : (N, N) float tensor — static k-NN adjacency A_init
    labels    : (N,) long tensor    — node class indices (for reordering)
    save_path : optional file path to save the figure
    """
    try:
        import seaborn as sns
    except ImportError:
        print("seaborn not available — install with: pip install seaborn")
        return

    N = A_dynamic.size(0)
    sort_idx   = torch.argsort(labels).cpu().numpy()
    A_dyn_np   = A_dynamic.cpu().detach().numpy()[sort_idx][:, sort_idx]
    A_stat_np  = A_static.cpu().detach().numpy()[sort_idx][:, sort_idx]
    labels_np  = labels.cpu().numpy()[sort_idx]

    # Build tick positions at class boundaries
    unique, counts = np.unique(labels_np, return_counts=True)
    boundaries = np.cumsum(counts)[:-1] - 0.5

    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    for ax, mat, title in zip(
            axes,
            [A_stat_np, A_dyn_np],
            ["Static k-NN Adjacency A_init", "Learned Dynamic Adjacency A'(L)"]):
        sns.heatmap(mat, ax=ax, cmap="Blues", vmin=0, vmax=1,
                    xticklabels=False, yticklabels=False,
                    cbar_kws={"shrink": 0.8})
        # Overlay class boundary lines
        for b in boundaries:
            ax.axhline(y=b, color="red", linewidth=0.8, linestyle="--", alpha=0.7)
            ax.axvline(x=b, color="red", linewidth=0.8, linestyle="--", alpha=0.7)
        ax.set_title(title, fontsize=11)
        ax.set_xlabel(f"N={N} nodes (sorted by class)")

    fig.suptitle("Adjacency Comparison: Static vs. Dynamic\n"
                 "(Block-diagonal = class-consistent topology; red lines = class boundaries)",
                 fontsize=11)
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=120, bbox_inches="tight")
    plt.show()


# ── Demo: run all plots on the toy model ─────────────────────────────────────

# Plot 1 — episode topology (coloured by class)
_labels_demo = torch.cat([torch.full((4,), c) for c in range(5)]).to(DEVICE)
plot_topology(_A_dyn, _labels_demo, title="Episode Topology (Layer 3)",
              save_path=str(SAVE_DIR / "topology_layer3.png"))

# Plot 2 — degree distribution
plot_degree_distribution(_A_dyn, "Degree Distribution (DEKAE)",
                         save_path=str(SAVE_DIR / "degree_dist.png"))

# Plots 3/3b/4/5 require training history & real data — run after train()
print("\nPlots 3/3b/4/5/6 require training history & real data — run after train().")
print("After training, call:")
print("  plot_density_curve(train_history)")
print("  plot_edge_ratio_curve(train_history)   ← NEW: monitors edge-loss effectiveness")
print("  plot_tsne(H_before, H_after, labels)")
print("  plot_degree_vs_difficulty(eval_model, test_sampler, n_way=5, k_shot=1, n_query=15)")
print("  plot_adjacency_heatmap(A_dyn, A_static, labels) ← plan §6.5 item 6")


NameError: name 'RESULTS_DIR' is not defined